# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ============================================================================
# KAGGLE NOTEBOOK: BRACS WSI FEATURE EXTRACTION (PARALLEL BATCH PROCESSING)
# ============================================================================

# Import all required libraries
import os
import numpy as np
import cv2
import kagglehub
import openslide
from skimage.color import rgb2hsv
from skimage.morphology import remove_small_holes, remove_small_objects
from skimage import feature
from skimage.feature import local_binary_pattern, hog, graycomatrix, graycoprops
from scipy import stats
from scipy.stats import skew, kurtosis
import pywt
import json
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

# Number of slides to process in parallel (adjust based on Kaggle performance)
PARALLEL_WORKERS = 4  # Kaggle has 4 CPU cores - process 4 slides at once
BATCH_SIZE = 8  # Process slides in batches (adjust if memory issues)

# ============================================================================
# STEP 1: CHOOSE WHICH DATASET TO PROCESS
# ============================================================================

print("="*80)
print("BRACS WSI FEATURE EXTRACTION - PARALLEL BATCH MODE")
print("="*80)

# SELECT ONE DATASET AT A TIME
dataset_to_process = "saadinn/bracs-wsi-group-mt-type-ic-01"  # ← CHANGE THIS FOR EACH RUN

print(f"\nDownloading dataset: {dataset_to_process}")
dataset_path = kagglehub.dataset_download(dataset_to_process)
dataset_name = dataset_to_process.split('/')[-1]
print(f"✓ Downloaded to: {dataset_path}")
print(f"✓ Dataset name: {dataset_name}")
print(f"✓ Parallel workers: {PARALLEL_WORKERS}")
print(f"✓ Batch size: {BATCH_SIZE}")

# ============================================================================
# STEP 2: TISSUE DETECTION FUNCTIONS
# ============================================================================

def build_tissue_mask(slide, level=4):
    """Creates binary mask identifying tissue regions at low resolution"""
    img = np.array(slide.read_region((0, 0), level, slide.level_dimensions[level]))[:, :, :3]
    hsv = rgb2hsv(img)
    saturation = hsv[:, :, 1]
    mask = saturation > 0.05
    mask = remove_small_objects(mask, min_size=500)
    mask = remove_small_holes(mask, area_threshold=500)
    return mask

def tissue_ratio(mask, x, y, patch_size, scale):
    """Calculates proportion of tissue in a patch location"""
    xs = int(x / scale)
    ys = int(y / scale)
    ps = int(patch_size / scale)
    patch_mask = mask[ys:ys+ps, xs:xs+ps]
    if patch_mask.size == 0:
        return 0.0
    return patch_mask.mean()

def preprocess_patch(patch):
    """Normalizes staining variations using histogram equalization"""
    patch = cv2.cvtColor(patch, cv2.COLOR_RGB2LAB)
    patch[:, :, 0] = cv2.equalizeHist(patch[:, :, 0])
    patch = cv2.cvtColor(patch, cv2.COLOR_LAB2RGB)
    return patch

# ============================================================================
# STEP 3: ENHANCED FEATURE EXTRACTORS
# ============================================================================

class EnhancedColorExtractor:
    """Extracts comprehensive color features from RGB images"""
    
    @staticmethod
    def extract_all_color_features(rgb_patch: np.ndarray) -> dict:
        features = {}
        
        # RGB channel features
        for idx, channel_name in enumerate(['R', 'G', 'B']):
            channel = rgb_patch[:, :, idx].flatten()
            features[f'{channel_name}_mean'] = float(np.mean(channel))
            features[f'{channel_name}_std'] = float(np.std(channel))
            features[f'{channel_name}_skewness'] = float(skew(channel))
            features[f'{channel_name}_kurtosis'] = float(kurtosis(channel))
            features[f'{channel_name}_min'] = float(np.min(channel))
            features[f'{channel_name}_max'] = float(np.max(channel))
            features[f'{channel_name}_range'] = float(np.max(channel) - np.min(channel))
            hist, _ = np.histogram(channel, bins=256, range=(0, 255))
            hist = hist / (hist.sum() + 1e-10)
            features[f'{channel_name}_entropy'] = float(-np.sum(hist * np.log2(hist + 1e-10)))
        
        # HSV features
        hsv = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2HSV)
        for idx, channel_name in enumerate(['H', 'S', 'V']):
            channel = hsv[:, :, idx].flatten()
            features[f'HSV_{channel_name}_mean'] = float(np.mean(channel))
            features[f'HSV_{channel_name}_std'] = float(np.std(channel))
        
        # LAB features
        lab = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2LAB)
        for idx, channel_name in enumerate(['L', 'A', 'B']):
            channel = lab[:, :, idx].flatten()
            features[f'LAB_{channel_name}_mean'] = float(np.mean(channel))
            features[f'LAB_{channel_name}_std'] = float(np.std(channel))
        
        return features


class EnhancedTextureExtractor:
    """Comprehensive texture feature extraction"""
    
    @staticmethod
    def extract_all_texture_features(gray_patch: np.ndarray) -> dict:
        features = {}
        
        # LBP features
        try:
            lbp = local_binary_pattern(gray_patch, 24, 3, method='uniform')
            features['LBP_mean'] = float(np.mean(lbp))
            features['LBP_std'] = float(np.std(lbp))
        except:
            features['LBP_mean'] = 0.0
            features['LBP_std'] = 0.0
        
        # Gabor features
        try:
            for freq_idx, frequency in enumerate([0.1, 0.2]):
                for theta_idx, theta in enumerate([0, np.pi/4]):
                    kernel = cv2.getGaborKernel((31, 31), 5.0, theta, 1.0/frequency, 0.5, 0)
                    filtered = cv2.filter2D(gray_patch, cv2.CV_32F, kernel)
                    features[f'Gabor_f{freq_idx}_t{theta_idx}_mean'] = float(np.mean(filtered))
                    features[f'Gabor_f{freq_idx}_t{theta_idx}_std'] = float(np.std(filtered))
        except:
            pass
        
        # GLCM features
        try:
            gray_quantized = (gray_patch / 32).astype(np.uint8)
            glcm = graycomatrix(gray_quantized, [1], [0], levels=8, symmetric=True, normed=True)
            features['GLCM_contrast'] = float(graycoprops(glcm, 'contrast')[0, 0])
            features['GLCM_homogeneity'] = float(graycoprops(glcm, 'homogeneity')[0, 0])
            features['GLCM_energy'] = float(graycoprops(glcm, 'energy')[0, 0])
        except:
            pass
        
        return features


class WaveletFeatureExtractor:
    """Wavelet transform features"""
    
    @staticmethod
    def extract_wavelet_features(gray_patch: np.ndarray) -> dict:
        features = {}
        
        try:
            coeffs = pywt.wavedec2(gray_patch, 'db4', level=2)
            
            # Approximation coefficients
            cA = coeffs[0]
            features['Wavelet_L0_LL_mean'] = float(np.mean(cA))
            features['Wavelet_L0_LL_std'] = float(np.std(cA))
            features['Wavelet_L0_LL_energy'] = float(np.sum(cA ** 2))
            
            # Detail coefficients
            for i in range(1, len(coeffs)):
                cH, cV, cD = coeffs[i]
                features[f'Wavelet_L{i}_LH_energy'] = float(np.sum(cH ** 2))
                features[f'Wavelet_L{i}_HL_energy'] = float(np.sum(cV ** 2))
                features[f'Wavelet_L{i}_HH_energy'] = float(np.sum(cD ** 2))
        except:
            pass
        
        return features


class ComprehensivePixelFeatureExtractor:
    """Combines all pixel-level feature extractors"""
    
    def __init__(self):
        self.color_extractor = EnhancedColorExtractor()
        self.texture_extractor = EnhancedTextureExtractor()
        self.wavelet_extractor = WaveletFeatureExtractor()
    
    def extract_all_features(self, rgb_patch: np.ndarray) -> dict:
        all_features = {}
        gray_patch = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
        all_features.update(self.color_extractor.extract_all_color_features(rgb_patch))
        all_features.update(self.texture_extractor.extract_all_texture_features(gray_patch))
        all_features.update(self.wavelet_extractor.extract_wavelet_features(gray_patch))
        return all_features

# ============================================================================
# STEP 4: PATCH-LEVEL FEATURE EXTRACTION
# ============================================================================

def extract_features_from_wsi(
    wsi_path,
    patch_size=256,
    target_level=1,
    tissue_threshold=0.3
):
    """Extracts features from all tissue patches in a WSI"""
    slide = openslide.OpenSlide(wsi_path)
    mask_level = slide.level_count - 1
    tissue_mask = build_tissue_mask(slide, mask_level)
    scale = slide.level_downsamples[mask_level]
    extractor = ComprehensivePixelFeatureExtractor()
    features_list = []
    width, height = slide.level_dimensions[target_level]
    step = patch_size
    
    # Count total patches
    total_patches = 0
    for y in range(0, height, step):
        for x in range(0, width, step):
            ratio = tissue_ratio(tissue_mask, x * slide.level_downsamples[target_level],
                                y * slide.level_downsamples[target_level], patch_size, scale)
            if ratio >= tissue_threshold:
                total_patches += 1
    
    # Extract patches (NO progress bar for parallel processing)
    for y in range(0, height, step):
        for x in range(0, width, step):
            ratio = tissue_ratio(tissue_mask, x * slide.level_downsamples[target_level],
                                y * slide.level_downsamples[target_level], patch_size, scale)
            if ratio < tissue_threshold:
                continue
            
            patch = slide.read_region(
                (int(x * slide.level_downsamples[target_level]),
                 int(y * slide.level_downsamples[target_level])),
                target_level, (patch_size, patch_size)
            )
            patch = np.array(patch)[:, :, :3]
            patch = preprocess_patch(patch)
            patch_features = extractor.extract_all_features(patch)
            patch_features['patch_x'] = x
            patch_features['patch_y'] = y
            patch_features['tissue_ratio'] = float(ratio)
            features_list.append(patch_features)
    
    slide.close()
    return features_list

# ============================================================================
# STEP 5: SLIDE-LEVEL AGGREGATION
# ============================================================================

def aggregate_patch_features_to_slide_level(patch_features_list, slide_name):
    """Converts patch-level features to slide-level features"""
    if not patch_features_list:
        return {}
    
    df = pd.DataFrame(patch_features_list)
    metadata_cols = ['patch_x', 'patch_y', 'tissue_ratio']
    feature_cols = [col for col in df.columns if col not in metadata_cols]
    
    slide_features = {}
    slide_features['slide_name'] = slide_name
    slide_features['num_patches'] = len(patch_features_list)
    
    # Aggregate each feature
    for feature in feature_cols:
        feature_values = df[feature].values
        slide_features[f'{feature}_mean'] = float(np.mean(feature_values))
        slide_features[f'{feature}_std'] = float(np.std(feature_values))
        slide_features[f'{feature}_min'] = float(np.min(feature_values))
        slide_features[f'{feature}_max'] = float(np.max(feature_values))
        slide_features[f'{feature}_median'] = float(np.median(feature_values))
        slide_features[f'{feature}_q25'] = float(np.percentile(feature_values, 25))
        slide_features[f'{feature}_q75'] = float(np.percentile(feature_values, 75))
    
    # Tissue coverage statistics
    slide_features['tissue_ratio_mean'] = float(df['tissue_ratio'].mean())
    slide_features['tissue_ratio_std'] = float(df['tissue_ratio'].std())
    
    return slide_features

# ============================================================================
# STEP 6: PARALLEL SLIDE PROCESSING FUNCTION
# ============================================================================

def process_single_slide(args):
    """
    Process one complete slide (designed for parallel execution)
    This function will be called by multiple workers simultaneously
    """
    slide_path, dataset_name, patch_size, target_level, tissue_threshold = args
    slide_name = os.path.basename(slide_path)
    
    try:
        # Extract PATCH-level features
        patch_features = extract_features_from_wsi(
            slide_path,
            patch_size=patch_size,
            target_level=target_level,
            tissue_threshold=tissue_threshold
        )
        
        if patch_features:
            # Aggregate to SLIDE-level
            slide_features = aggregate_patch_features_to_slide_level(
                patch_features, 
                f"{dataset_name}_{slide_name}"
            )
            
            return {
                'success': True,
                'slide_path': slide_path,
                'slide_name': slide_name,
                'features': slide_features,
                'num_patches': len(patch_features),
                'error': None
            }
        else:
            return {
                'success': False,
                'slide_path': slide_path,
                'slide_name': slide_name,
                'features': None,
                'num_patches': 0,
                'error': 'No tissue patches found'
            }
            
    except Exception as e:
        return {
            'success': False,
            'slide_path': slide_path,
            'slide_name': slide_name,
            'features': None,
            'num_patches': 0,
            'error': str(e)
        }

# ============================================================================
# STEP 7: CHECKPOINT SYSTEM
# ============================================================================

def load_checkpoint(dataset_name):
    """Load checkpoint for specific dataset"""
    checkpoint_file = f'/kaggle/working/checkpoint_{dataset_name}.json'
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            return set(json.load(f))
    return set()

def save_checkpoint(processed_slides, dataset_name):
    """Save checkpoint for specific dataset"""
    checkpoint_file = f'/kaggle/working/checkpoint_{dataset_name}.json'
    with open(checkpoint_file, 'w') as f:
        json.dump(list(processed_slides), f)

# ============================================================================
# STEP 8: MAIN PROCESSING LOOP WITH PARALLEL BATCHES
# ============================================================================

print("\n" + "="*80)
print("STARTING PARALLEL BATCH FEATURE EXTRACTION")
print("="*80)

# Create output directory
output_path = f'/kaggle/working/features_{dataset_name}/'
os.makedirs(output_path, exist_ok=True)

# Load checkpoint
processed_slides = load_checkpoint(dataset_name)
print(f"Already processed: {len(processed_slides)} slides")

# Find all slides
slides = []
for root, dirs, files in os.walk(dataset_path):
    slides.extend([os.path.join(root, f) for f in files 
                  if f.endswith(('.svs', '.tif', '.tiff', '.ndpi'))])

# Filter out already processed slides
slides_to_process = [s for s in slides if s not in processed_slides]

print(f"Total slides in dataset: {len(slides)}")
print(f"Slides remaining to process: {len(slides_to_process)}")

# Storage for slide-level features
all_slides_features = []

# Process slides in batches with parallel workers
total_processed = 0

for batch_start in range(0, len(slides_to_process), BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, len(slides_to_process))
    batch_slides = slides_to_process[batch_start:batch_end]
    
    print(f"\n{'='*80}")
    print(f"PROCESSING BATCH {batch_start//BATCH_SIZE + 1}")
    print(f"Slides {batch_start + 1}-{batch_end} of {len(slides_to_process)}")
    print(f"Processing {len(batch_slides)} slides with {PARALLEL_WORKERS} parallel workers")
    print(f"{'='*80}")
    
    # Prepare arguments for parallel processing
    slide_args = [
        (slide_path, dataset_name, 256, 1, 0.3)
        for slide_path in batch_slides
    ]
    
    # Process batch in parallel
    with ProcessPoolExecutor(max_workers=PARALLEL_WORKERS) as executor:
        # Submit all slides in batch
        futures = {executor.submit(process_single_slide, args): args[0] 
                  for args in slide_args}
        
        # Collect results as they complete
        for future in as_completed(futures):
            slide_path = futures[future]
            
            try:
                result = future.result()
                
                if result['success']:
                    print(f"  ✓ {result['slide_name']}: {result['num_patches']} patches")
                    all_slides_features.append(result['features'])
                    processed_slides.add(result['slide_path'])
                    total_processed += 1
                else:
                    print(f"  ✗ {result['slide_name']}: {result['error']}")
                    
            except Exception as e:
                print(f"  ✗ Error with {os.path.basename(slide_path)}: {str(e)}")
    
    # Save checkpoint after each batch
    save_checkpoint(processed_slides, dataset_name)
    
    # Save temporary results
    if all_slides_features:
        temp_df = pd.DataFrame(all_slides_features)
        temp_df.to_csv(f'{output_path}/slide_features_temp.csv', index=False)
        print(f"\n  💾 Batch checkpoint: {total_processed} slides completed")

# ============================================================================
# STEP 9: SAVE FINAL RESULTS
# ============================================================================

print("\n" + "="*80)
print("SAVING FINAL RESULTS")
print("="*80)

if all_slides_features:
    # Convert to DataFrame
    final_df = pd.DataFrame(all_slides_features)
    
    print(f"\nFinal dataset shape: {final_df.shape}")
    print(f"  - Rows (slides): {final_df.shape[0]}")
    print(f"  - Columns (features): {final_df.shape[1]}")
    
    # Save as CSV
    csv_path = f'{output_path}/{dataset_name}_slide_features.csv'
    final_df.to_csv(csv_path, index=False)
    print(f"✓ Saved CSV to: {csv_path}")
    
    # Save as compressed numpy
    npz_path = f'{output_path}/{dataset_name}_slide_features.npz'
    np.savez_compressed(
        npz_path,
        slide_names=final_df['slide_name'].values,
        features=final_df.drop('slide_name', axis=1).values,
        feature_names=final_df.drop('slide_name', axis=1).columns.values
    )
    print(f"✓ Saved NPZ to: {npz_path}")
    
    # Show sample
    print("\n" + "="*80)
    print("SAMPLE OUTPUT")
    print("="*80)
    print(final_df.head())
    
    print("\n" + "="*80)
    print("FEATURE EXTRACTION COMPLETE!")
    print("="*80)
    print(f"Dataset: {dataset_name}")
    print(f"Total slides processed: {len(all_slides_features)}")
    print(f"Features per slide: {final_df.shape[1]}")
    print(f"Output directory: {output_path}")
    print(f"Parallel workers used: {PARALLEL_WORKERS}")
    print(f"Batch size: {BATCH_SIZE}")
else:
    print("⚠ No slides were successfully processed!")

print("\n" + "="*80)
print("NEXT STEPS:")
print("="*80)
print("1. Download the CSV file")
print("2. Change the dataset name at the top of this code:")
print(f'   dataset_to_process = "saadinn/bracs-wsi-group-NEXT-DATASET"')
print("3. Run again for the next dataset")
print("4. Repeat for all 9 datasets")
print("="*80)


# Object Level extraction

# ============================================================================
# KAGGLE NOTEBOOK: BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION (OPTIMIZED CPU)
# ============================================================================

# Import all required libraries
import os
import numpy as np
import cv2
import kagglehub
import openslide
from skimage.color import rgb2hsv
from skimage.morphology import remove_small_holes, remove_small_objects, disk, erosion, dilation
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from scipy.spatial.distance import cdist
from scipy.stats import skew, kurtosis
import json
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

PARALLEL_WORKERS = 4  # Kaggle has 4 CPU cores
BATCH_SIZE = 8

# ============================================================================
# STEP 1: CHOOSE WHICH DATASET TO PROCESS
# ============================================================================

print("="*80)
print("BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION - OPTIMIZED CPU VERSION")
print("="*80)

dataset_to_process = "saadinn/bracs-wsi-group-mt-type-ic-01"

print(f"\nDownloading dataset: {dataset_to_process}")
dataset_path = kagglehub.dataset_download(dataset_to_process)
dataset_name = dataset_to_process.split('/')[-1]
print(f"✓ Downloaded to: {dataset_path}")
print(f"✓ Dataset name: {dataset_name}")
print(f"✓ Parallel workers: {PARALLEL_WORKERS}")
print(f"✓ Batch size: {BATCH_SIZE}")

# ============================================================================
# STEP 2: TISSUE DETECTION FUNCTIONS
# ============================================================================

def build_tissue_mask(slide, level=4):
    """Creates binary mask identifying tissue regions at low resolution"""
    img = np.array(slide.read_region((0, 0), level, slide.level_dimensions[level]))[:, :, :3]
    hsv = rgb2hsv(img)
    saturation = hsv[:, :, 1]
    mask = saturation > 0.05
    mask = remove_small_objects(mask, min_size=500)
    mask = remove_small_holes(mask, area_threshold=500)
    return mask

def tissue_ratio(mask, x, y, patch_size, scale):
    """Calculates proportion of tissue in a patch location"""
    xs = int(x / scale)
    ys = int(y / scale)
    ps = int(patch_size / scale)
    patch_mask = mask[ys:ys+ps, xs:xs+ps]
    if patch_mask.size == 0:
        return 0.0
    return patch_mask.mean()

# ============================================================================
# STEP 3: NUCLEUS SEGMENTATION
# ============================================================================

class NucleusSegmenter:
    """Segments nuclei from H&E stained histopathology images"""
    
    @staticmethod
    def extract_hematoxylin_channel(rgb_patch):
        """Extract hematoxylin channel using color deconvolution"""
        # Normalize RGB to [0, 1]
        rgb_norm = rgb_patch.astype(np.float32) / 255.0
        rgb_norm = np.maximum(rgb_norm, 1e-6)
        
        # Convert to optical density (OD)
        od = -np.log(rgb_norm)
        
        # H&E stain matrix
        HE_matrix = np.array([
            [0.65, 0.70, 0.29],
            [0.07, 0.99, 0.11],
            [0.27, 0.57, 0.78]
        ])
        
        # Deconvolve to get stain concentrations
        od_flat = od.reshape(-1, 3)
        stains = np.linalg.lstsq(HE_matrix.T, od_flat.T, rcond=None)[0].T
        hematoxylin = stains[:, 0].reshape(rgb_patch.shape[:2])
        
        # Normalize to [0, 255]
        hematoxylin = np.clip(hematoxylin, 0, None)
        hematoxylin = (hematoxylin / (hematoxylin.max() + 1e-6) * 255).astype(np.uint8)
        
        return hematoxylin
    
    @staticmethod
    def segment_nuclei(rgb_patch):
        """Segments nuclei using multiple methods for robustness"""
        try:
            hematoxylin = NucleusSegmenter.extract_hematoxylin_channel(rgb_patch)
            _, nucleus_mask = cv2.threshold(hematoxylin, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            nucleus_mask = nucleus_mask > 0
        except:
            gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
            gray_inv = 255 - gray
            _, nucleus_mask = cv2.threshold(gray_inv, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            nucleus_mask = nucleus_mask > 0
        
        nucleus_mask = remove_small_objects(nucleus_mask, min_size=15)
        nucleus_mask = remove_small_holes(nucleus_mask, area_threshold=15)
        
        if not nucleus_mask.any():
            return np.zeros(rgb_patch.shape[:2], dtype=int)
        
        nucleus_mask = erosion(nucleus_mask, disk(1))
        nucleus_mask = dilation(nucleus_mask, disk(2))
        
        distance = ndi.distance_transform_edt(nucleus_mask)
        coords = peak_local_max(distance, min_distance=3, labels=nucleus_mask, exclude_border=False)
        
        if len(coords) == 0:
            labeled_nuclei = label(nucleus_mask)
            return labeled_nuclei
        
        mask_centers = np.zeros(distance.shape, dtype=bool)
        mask_centers[tuple(coords.T)] = True
        markers = label(mask_centers)
        labeled_nuclei = watershed(-distance, markers, mask=nucleus_mask)
        
        return labeled_nuclei

# ============================================================================
# STEP 4: OBJECT-LEVEL FEATURE EXTRACTORS
# ============================================================================

class NucleusFeatureExtractor:
    """Extracts morphological and intensity features from individual nuclei"""
    
    @staticmethod
    def extract_nucleus_features(region, intensity_image):
        """Extract features from a single nucleus region"""
        features = {}
        
        features['area'] = float(region.area)
        features['perimeter'] = float(region.perimeter)
        features['eccentricity'] = float(region.eccentricity)
        features['solidity'] = float(region.solidity)
        features['extent'] = float(region.extent)
        features['orientation'] = float(region.orientation)
        
        if region.perimeter > 0:
            features['circularity'] = float(4 * np.pi * region.area / (region.perimeter ** 2))
        else:
            features['circularity'] = 0.0
        
        features['major_axis_length'] = float(region.major_axis_length)
        features['minor_axis_length'] = float(region.minor_axis_length)
        
        if region.minor_axis_length > 0:
            features['aspect_ratio'] = float(region.major_axis_length / region.minor_axis_length)
        else:
            features['aspect_ratio'] = 0.0
        
        bbox = region.bbox
        features['bbox_width'] = float(bbox[3] - bbox[1])
        features['bbox_height'] = float(bbox[2] - bbox[0])
        features['bbox_area'] = float(features['bbox_width'] * features['bbox_height'])
        
        nucleus_pixels = intensity_image[region.coords[:, 0], region.coords[:, 1]]
        features['intensity_mean'] = float(np.mean(nucleus_pixels))
        features['intensity_std'] = float(np.std(nucleus_pixels))
        features['intensity_min'] = float(np.min(nucleus_pixels))
        features['intensity_max'] = float(np.max(nucleus_pixels))
        features['intensity_range'] = float(np.max(nucleus_pixels) - np.min(nucleus_pixels))
        features['intensity_median'] = float(np.median(nucleus_pixels))
        features['intensity_skewness'] = float(skew(nucleus_pixels))
        features['intensity_kurtosis'] = float(kurtosis(nucleus_pixels))
        
        return features


class SpatialFeatureExtractor:
    """Extracts spatial arrangement features from nuclei populations"""
    
    @staticmethod
    def extract_spatial_features(regions):
        """Extract features describing spatial distribution of nuclei"""
        features = {}
        
        if len(regions) == 0:
            return features
        
        centroids = np.array([region.centroid for region in regions])
        
        features['nucleus_count'] = len(regions)
        total_area = sum([region.area for region in regions])
        features['total_nucleus_area'] = float(total_area)
        
        if len(centroids) > 1:
            distances = cdist(centroids, centroids)
            np.fill_diagonal(distances, np.inf)
            
            nn_distances = np.min(distances, axis=1)
            features['nn_distance_mean'] = float(np.mean(nn_distances))
            features['nn_distance_std'] = float(np.std(nn_distances))
            features['nn_distance_min'] = float(np.min(nn_distances))
            features['nn_distance_max'] = float(np.max(nn_distances))
            features['nn_distance_median'] = float(np.median(nn_distances))
            
            upper_triangle = distances[np.triu_indices_from(distances, k=1)]
            features['pairwise_distance_mean'] = float(np.mean(upper_triangle))
            features['pairwise_distance_std'] = float(np.std(upper_triangle))
            features['pairwise_distance_min'] = float(np.min(upper_triangle))
            features['pairwise_distance_max'] = float(np.max(upper_triangle))
        
        if len(centroids) > 2:
            features['centroid_x_variance'] = float(np.var(centroids[:, 1]))
            features['centroid_y_variance'] = float(np.var(centroids[:, 0]))
            features['centroid_x_range'] = float(np.max(centroids[:, 1]) - np.min(centroids[:, 1]))
            features['centroid_y_range'] = float(np.max(centroids[:, 0]) - np.min(centroids[:, 0]))
        
        return features


class GraphFeatureExtractor:
    """Extracts graph-based features from nucleus spatial relationships"""
    
    @staticmethod
    def build_delaunay_graph(centroids, max_edge_length=100):
        """Build graph connecting nearby nuclei"""
        from scipy.spatial import Delaunay
        
        if len(centroids) < 3:
            return None, None
        
        try:
            tri = Delaunay(centroids)
            edges = set()
            for simplex in tri.simplices:
                for i in range(3):
                    edge = tuple(sorted([simplex[i], simplex[(i+1)%3]]))
                    dist = np.linalg.norm(centroids[edge[0]] - centroids[edge[1]])
                    if dist < max_edge_length:
                        edges.add(edge)
            return tri, list(edges)
        except:
            return None, None
    
    @staticmethod
    def extract_graph_features(regions):
        """Extract graph-based topological features"""
        features = {}
        
        if len(regions) < 3:
            return features
        
        centroids = np.array([region.centroid for region in regions])
        tri, edges = GraphFeatureExtractor.build_delaunay_graph(centroids)
        
        if edges is None or len(edges) == 0:
            return features
        
        features['num_edges'] = len(edges)
        features['edge_density'] = float(len(edges) / (len(regions) * (len(regions) - 1) / 2))
        
        degree_count = {}
        for edge in edges:
            degree_count[edge[0]] = degree_count.get(edge[0], 0) + 1
            degree_count[edge[1]] = degree_count.get(edge[1], 0) + 1
        
        if degree_count:
            degrees = list(degree_count.values())
            features['degree_mean'] = float(np.mean(degrees))
            features['degree_std'] = float(np.std(degrees))
            features['degree_max'] = float(np.max(degrees))
            features['degree_min'] = float(np.min(degrees))
        
        edge_lengths = [np.linalg.norm(centroids[e[0]] - centroids[e[1]]) for e in edges]
        features['edge_length_mean'] = float(np.mean(edge_lengths))
        features['edge_length_std'] = float(np.std(edge_lengths))
        features['edge_length_min'] = float(np.min(edge_lengths))
        features['edge_length_max'] = float(np.max(edge_lengths))
        
        return features


class ComprehensiveObjectFeatureExtractor:
    """Combines all object-level feature extractors"""
    
    def __init__(self):
        self.nucleus_extractor = NucleusFeatureExtractor()
        self.spatial_extractor = SpatialFeatureExtractor()
        self.graph_extractor = GraphFeatureExtractor()
    
    def extract_all_features(self, rgb_patch):
        """Extract all object-level features from a patch"""
        all_features = {}
        
        labeled_nuclei = NucleusSegmenter.segment_nuclei(rgb_patch)
        gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
        regions = regionprops(labeled_nuclei, intensity_image=gray)
        regions = [r for r in regions if r.area >= 10]
        
        all_features['nucleus_count'] = len(regions)
        all_features['total_nucleus_area'] = float(sum([r.area for r in regions]) if regions else 0.0)
        
        if len(regions) > 0:
            nucleus_features_list = []
            for region in regions:
                try:
                    nuc_feat = self.nucleus_extractor.extract_nucleus_features(region, gray)
                    nucleus_features_list.append(nuc_feat)
                except:
                    continue
            
            if nucleus_features_list:
                df = pd.DataFrame(nucleus_features_list)
                for col in df.columns:
                    all_features[f'nucleus_{col}_mean'] = float(df[col].mean())
                    all_features[f'nucleus_{col}_std'] = float(df[col].std())
                    all_features[f'nucleus_{col}_min'] = float(df[col].min())
                    all_features[f'nucleus_{col}_max'] = float(df[col].max())
                    all_features[f'nucleus_{col}_median'] = float(df[col].median())
        
        if len(regions) > 0:
            spatial_features = self.spatial_extractor.extract_spatial_features(regions)
            all_features.update(spatial_features)
        
        if len(regions) >= 3:
            graph_features = self.graph_extractor.extract_graph_features(regions)
            all_features.update(graph_features)
        
        return all_features

# ============================================================================
# STEP 5: PATCH-LEVEL FEATURE EXTRACTION
# ============================================================================

def extract_object_features_from_wsi(wsi_path, patch_size=256, target_level=1, tissue_threshold=0.3):
    """Extracts object-level features from all tissue patches in a WSI"""
    slide = openslide.OpenSlide(wsi_path)
    mask_level = slide.level_count - 1
    tissue_mask = build_tissue_mask(slide, mask_level)
    scale = slide.level_downsamples[mask_level]
    extractor = ComprehensiveObjectFeatureExtractor()
    features_list = []
    width, height = slide.level_dimensions[target_level]
    step = patch_size
    
    for y in range(0, height, step):
        for x in range(0, width, step):
            ratio = tissue_ratio(tissue_mask, x * slide.level_downsamples[target_level],
                                y * slide.level_downsamples[target_level], patch_size, scale)
            if ratio < tissue_threshold:
                continue
            
            patch = slide.read_region(
                (int(x * slide.level_downsamples[target_level]),
                 int(y * slide.level_downsamples[target_level])),
                target_level, (patch_size, patch_size)
            )
            patch = np.array(patch)[:, :, :3]
            
            try:
                patch_features = extractor.extract_all_features(patch)
                patch_features['patch_x'] = x
                patch_features['patch_y'] = y
                patch_features['tissue_ratio'] = float(ratio)
                features_list.append(patch_features)
            except Exception as e:
                continue
    
    slide.close()
    return features_list

# ============================================================================
# STEP 6: SLIDE-LEVEL AGGREGATION
# ============================================================================

def aggregate_patch_features_to_slide_level(patch_features_list, slide_name):
    """Converts patch-level features to slide-level features with proper NaN handling"""
    if not patch_features_list:
        return {}
    
    df = pd.DataFrame(patch_features_list)
    metadata_cols = ['patch_x', 'patch_y', 'tissue_ratio']
    feature_cols = [col for col in df.columns if col not in metadata_cols]
    
    slide_features = {}
    slide_features['slide_name'] = slide_name
    slide_features['num_patches'] = len(patch_features_list)
    
    for feature in feature_cols:
        feature_values = df[feature].dropna().values
        
        if len(feature_values) > 0:
            slide_features[f'{feature}_mean'] = float(np.mean(feature_values))
            slide_features[f'{feature}_std'] = float(np.std(feature_values))
            slide_features[f'{feature}_min'] = float(np.min(feature_values))
            slide_features[f'{feature}_max'] = float(np.max(feature_values))
            slide_features[f'{feature}_median'] = float(np.median(feature_values))
            slide_features[f'{feature}_q25'] = float(np.percentile(feature_values, 25))
            slide_features[f'{feature}_q75'] = float(np.percentile(feature_values, 75))
        else:
            slide_features[f'{feature}_mean'] = 0.0
            slide_features[f'{feature}_std'] = 0.0
            slide_features[f'{feature}_min'] = 0.0
            slide_features[f'{feature}_max'] = 0.0
            slide_features[f'{feature}_median'] = 0.0
            slide_features[f'{feature}_q25'] = 0.0
            slide_features[f'{feature}_q75'] = 0.0
    
    slide_features['tissue_ratio_mean'] = float(df['tissue_ratio'].mean())
    slide_features['tissue_ratio_std'] = float(df['tissue_ratio'].std())
    
    return slide_features

# ============================================================================
# STEP 7: PARALLEL SLIDE PROCESSING
# ============================================================================

def process_single_slide(args):
    """Process one complete slide"""
    slide_path, dataset_name, patch_size, target_level, tissue_threshold = args
    slide_name = os.path.basename(slide_path)
    
    try:
        patch_features = extract_object_features_from_wsi(
            slide_path, patch_size, target_level, tissue_threshold
        )
        
        if patch_features:
            slide_features = aggregate_patch_features_to_slide_level(
                patch_features, f"{dataset_name}_{slide_name}"
            )
            
            return {
                'success': True,
                'slide_path': slide_path,
                'slide_name': slide_name,
                'features': slide_features,
                'num_patches': len(patch_features),
                'error': None
            }
        else:
            return {
                'success': False,
                'slide_path': slide_path,
                'slide_name': slide_name,
                'features': None,
                'num_patches': 0,
                'error': 'No tissue patches found'
            }
    except Exception as e:
        return {
            'success': False,
            'slide_path': slide_path,
            'slide_name': slide_name,
            'features': None,
            'num_patches': 0,
            'error': str(e)
        }

# ============================================================================
# STEP 8: CHECKPOINT SYSTEM
# ============================================================================

def load_checkpoint(dataset_name):
    checkpoint_file = f'/kaggle/working/checkpoint_object_{dataset_name}.json'
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            return set(json.load(f))
    return set()

def save_checkpoint(processed_slides, dataset_name):
    checkpoint_file = f'/kaggle/working/checkpoint_object_{dataset_name}.json'
    with open(checkpoint_file, 'w') as f:
        json.dump(list(processed_slides), f)

# ============================================================================
# STEP 9: MAIN PROCESSING LOOP
# ============================================================================

print("\n" + "="*80)
print("STARTING PARALLEL BATCH OBJECT-LEVEL FEATURE EXTRACTION")
print("="*80)

output_path = f'/kaggle/working/object_features_{dataset_name}/'
os.makedirs(output_path, exist_ok=True)

processed_slides = load_checkpoint(dataset_name)
print(f"Already processed: {len(processed_slides)} slides")

slides = []
for root, dirs, files in os.walk(dataset_path):
    slides.extend([os.path.join(root, f) for f in files 
                  if f.endswith(('.svs', '.tif', '.tiff', '.ndpi'))])

slides_to_process = [s for s in slides if s not in processed_slides]

print(f"Total slides in dataset: {len(slides)}")
print(f"Slides remaining to process: {len(slides_to_process)}")

all_slides_features = []
total_processed = 0

for batch_start in range(0, len(slides_to_process), BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, len(slides_to_process))
    batch_slides = slides_to_process[batch_start:batch_end]
    
    print(f"\n{'='*80}")
    print(f"PROCESSING BATCH {batch_start//BATCH_SIZE + 1}")
    print(f"Slides {batch_start + 1}-{batch_end} of {len(slides_to_process)}")
    print(f"Processing {len(batch_slides)} slides with {PARALLEL_WORKERS} parallel workers")
    print(f"{'='*80}")
    
    slide_args = [(slide_path, dataset_name, 256, 1, 0.3) for slide_path in batch_slides]
    
    with ProcessPoolExecutor(max_workers=PARALLEL_WORKERS) as executor:
        futures = {executor.submit(process_single_slide, args): args[0] for args in slide_args}
        
        for future in as_completed(futures):
            slide_path = futures[future]
            
            try:
                result = future.result()
                
                if result['success']:
                    print(f"  ✓ {result['slide_name']}: {result['num_patches']} patches")
                    all_slides_features.append(result['features'])
                    processed_slides.add(result['slide_path'])
                    total_processed += 1
                else:
                    print(f"  ✗ {result['slide_name']}: {result['error']}")
            except Exception as e:
                print(f"  ✗ Error with {os.path.basename(slide_path)}: {str(e)}")
    
    save_checkpoint(processed_slides, dataset_name)
    
    if all_slides_features:
        temp_df = pd.DataFrame(all_slides_features)
        temp_df.to_csv(f'{output_path}/slide_object_features_temp.csv', index=False)
        print(f"\n  💾 Batch checkpoint: {total_processed} slides completed")

# ============================================================================
# STEP 10: SAVE FINAL RESULTS
# ============================================================================

print("\n" + "="*80)
print("SAVING FINAL RESULTS")
print("="*80)

if all_slides_features:
    final_df = pd.DataFrame(all_slides_features)
    
    print(f"\nFinal dataset shape: {final_df.shape}")
    print(f"  - Rows (slides): {final_df.shape[0]}")
    print(f"  - Columns (features): {final_df.shape[1]}")
    
    print("\n" + "="*80)
    print("NUCLEUS DETECTION DIAGNOSTICS")
    print("="*80)
    
    nucleus_count_cols = [col for col in final_df.columns if 'nucleus_count' in col.lower()]
    if nucleus_count_cols:
        for col in nucleus_count_cols:
            mean_count = final_df[col].mean()
            print(f"{col}: mean = {mean_count:.2f}")
            zero_count = (final_df[col] == 0).sum()
            print(f"  Slides with 0 nuclei: {zero_count}/{len(final_df)} ({100*zero_count/len(final_df):.1f}%)")
    
    nan_counts = final_df.isna().sum()
    cols_with_nans = nan_counts[nan_counts > 0]
    if len(cols_with_nans) > 0:
        print(f"\nColumns with NaN values: {len(cols_with_nans)}/{len(final_df.columns)}")
        print(f"Most NaN values: {nan_counts.max()} ({100*nan_counts.max()/len(final_df):.1f}% of slides)")
        print("\nTop 10 columns with NaNs:")
        print(nan_counts.nlargest(10))
    else:
        print("\n✓ No NaN values found! All features properly computed.")
    
    zero_cols = (final_df == 0).all()
    all_zero_cols = zero_cols[zero_cols].index.tolist()
    if all_zero_cols:
        print(f"\nColumns with all zeros: {len(all_zero_cols)}")
        if len(all_zero_cols) <= 20:
            print(f"Zero columns: {all_zero_cols}")
    
    print("="*80)
    
    csv_path = f'{output_path}/{dataset_name}_slide_object_features.csv'
    final_df.to_csv(csv_path, index=False)
    print(f"\n✓ Saved CSV to: {csv_path}")
    
    npz_path = f'{output_path}/{dataset_name}_slide_object_features.npz'
    np.savez_compressed(
        npz_path,
        slide_names=final_df['slide_name'].values,
        features=final_df.drop('slide_name', axis=1).values,
        feature_names=final_df.drop('slide_name', axis=1).columns.values
    )
    print(f"✓ Saved NPZ to: {npz_path}")
    
    print("\n" + "="*80)
    print("SAMPLE OUTPUT")
    print("="*80)
    print(final_df.head())
    
    print("\n" + "="*80)
    print("OBJECT-LEVEL FEATURE EXTRACTION COMPLETE!")
    print("="*80)
    print(f"Dataset: {dataset_name}")
    print(f"Total slides processed: {len(all_slides_features)}")
    print(f"Features per slide: {final_df.shape[1]}")
else:
    print("⚠ No slides were successfully processed!")

print("\n" + "="*80)
print("NEXT STEPS:")
print("="*80)
print("1. Download the CSV file")
print("2. Change dataset_to_process for next dataset")
print("3. Repeat for all datasets")
print("="*80)


# ============================================================================
# STEP 11: SAVE OUTPUT AS KAGGLE DATASET
# ============================================================================

from kaggle import api
import shutil

print("\n" + "="*80)
print("CREATING KAGGLE DATASET FOR OUTPUT")
print("="*80)

# Create dataset metadata
dataset_slug = f"object-features-{dataset_name.replace('_', '-')}"
metadata = {
    "title": f"Object Features - {dataset_name}",
    "id": f"onyekamuoka/{dataset_slug}",
    "licenses": [{"name": "CC0-1.0"}]
}

metadata_path = f'{output_path}/dataset-metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

try:
    # Create new dataset (first time)
    api.dataset_create_new(
        folder=output_path,
        dir_mode='zip',
        convert_to_csv=False,
        public=False
    )
    print(f"✓ Created new dataset: {dataset_slug}")
except:
    # Update existing dataset (subsequent runs)
    api.dataset_create_version(
        folder=output_path,
        version_notes=f"Updated features for {dataset_name}",
        dir_mode='zip',
        convert_to_csv=False
    )
    print(f"✓ Updated existing dataset: {dataset_slug}")

print(f"\n✓ Dataset available at: https://www.kaggle.com/datasets/onyekamuoka/{dataset_slug}")
print("  Download anytime from Kaggle!")

# Object Level

# ============================================================================
# KAGGLE NOTEBOOK: BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION (SLICED RUNS)
# ============================================================================
#
# HOW TO USE:
#   This notebook processes a specific slice of slides to stay within
#   Kaggle's 12-hour time limit. Run it multiple times, changing only
#   SLIDE_START_INDEX and SLIDE_END_INDEX each time.
#
#   Recommended splits for UDH (74 slides, 31 already done):
#   ┌─────────┬─────────────────────┬──────────────────┐
#   │  Run    │  Slide Range        │  Change To       │
#   ├─────────┼─────────────────────┼──────────────────┤
#   │  Run A  │  slides 32 – 47    │  START=32 END=48 │
#   │  Run B  │  slides 48 – 63    │  START=48 END=64 │
#   │  Run C  │  slides 64 – 74    │  START=64 END=74 │
#   └─────────┴─────────────────────┴──────────────────┘
#
#   After all runs complete, use bracs_wsi_merge.py to combine the CSVs.
#
# ============================================================================

import os
import numpy as np
import cv2
import kagglehub
import openslide
from skimage.color import rgb2hsv
from skimage.morphology import remove_small_holes, remove_small_objects, disk, dilation
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from scipy.spatial.distance import cdist
from scipy.stats import skew, kurtosis
import json
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# *** CHANGE THESE TWO NUMBERS EACH RUN ***
# ============================================================================

SLIDE_START_INDEX = 0   # first slide to process (0-based index into sorted slide list)
SLIDE_END_INDEX   = 15   # process up to but NOT including this index

# ============================================================================
# CONFIGURATION (leave these unchanged across all runs)
# ============================================================================

PARALLEL_WORKERS = 4
BATCH_SIZE = 8
dataset_to_process = "saadinn/bracs-wsi-group-mt-type-ic-01"

# ============================================================================
# SETUP
# ============================================================================

print("="*80)
print("BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION - SLICED RUN")
print("="*80)
print(f"\nSlide slice: index {SLIDE_START_INDEX} → {SLIDE_END_INDEX - 1}")
print(f"Downloading dataset: {dataset_to_process}")

dataset_path = kagglehub.dataset_download(dataset_to_process)
dataset_name = dataset_to_process.split('/')[-1]

print(f"✓ Downloaded to: {dataset_path}")
print(f"✓ Dataset name:  {dataset_name}")

# ============================================================================
# STEP 2: TISSUE DETECTION
# ============================================================================

def build_tissue_mask(slide):
    mask_level = slide.level_count - 1
    img = np.array(
        slide.read_region((0, 0), mask_level, slide.level_dimensions[mask_level])
    )[:, :, :3]

    hsv = rgb2hsv(img)
    saturation = hsv[:, :, 1]
    value = hsv[:, :, 2]

    mask = (saturation > 0.03) & (value < 0.95)
    mask = remove_small_objects(mask, min_size=300)
    mask = remove_small_holes(mask, area_threshold=300)
    return mask, mask_level


def tissue_ratio(mask, x_level0, y_level0, patch_size_level0, scale):
    xs = int(x_level0 / scale)
    ys = int(y_level0 / scale)
    ps = max(int(patch_size_level0 / scale), 1)
    patch_mask = mask[ys:ys + ps, xs:xs + ps]
    if patch_mask.size == 0:
        return 0.0
    return float(patch_mask.mean())

# ============================================================================
# STEP 3: NUCLEUS SEGMENTATION
# ============================================================================

class NucleusSegmenter:

    @staticmethod
    def extract_hematoxylin_channel(rgb_patch):
        rgb_norm = rgb_patch.astype(np.float32) / 255.0
        rgb_norm = np.clip(rgb_norm, 1e-6, 1.0)
        od = -np.log(rgb_norm)

        HE_matrix = np.array([
            [0.65, 0.70, 0.29],
            [0.07, 0.99, 0.11],
            [0.27, 0.57, 0.78]
        ])

        od_flat = od.reshape(-1, 3)
        stains = np.linalg.lstsq(HE_matrix.T, od_flat.T, rcond=None)[0].T
        hematoxylin = stains[:, 0].reshape(rgb_patch.shape[:2])
        hematoxylin = np.clip(hematoxylin, 0, None)
        hematoxylin = (hematoxylin / (hematoxylin.max() + 1e-6) * 255).astype(np.uint8)
        return hematoxylin

    @staticmethod
    def segment_nuclei(rgb_patch):
        try:
            hematoxylin = NucleusSegmenter.extract_hematoxylin_channel(rgb_patch)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            hematoxylin_eq = clahe.apply(hematoxylin)

            _, nucleus_mask_otsu = cv2.threshold(
                hematoxylin_eq, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
            )

            otsu_ratio = nucleus_mask_otsu.mean() / 255.0
            if otsu_ratio < 0.05 or otsu_ratio > 0.70:
                nucleus_mask = cv2.adaptiveThreshold(
                    hematoxylin_eq, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY,
                    blockSize=31,
                    C=-2
                )
                nucleus_mask = nucleus_mask > 0
            else:
                nucleus_mask = nucleus_mask_otsu > 0

        except Exception:
            gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
            gray_inv = 255 - gray
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            gray_inv_eq = clahe.apply(gray_inv)
            nucleus_mask = cv2.adaptiveThreshold(
                gray_inv_eq, 255,
                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY,
                blockSize=31,
                C=-2
            )
            nucleus_mask = nucleus_mask > 0

        nucleus_mask = remove_small_objects(nucleus_mask, min_size=8)
        nucleus_mask = remove_small_holes(nucleus_mask, area_threshold=8)

        if not nucleus_mask.any():
            return np.zeros(rgb_patch.shape[:2], dtype=int)

        nucleus_mask = dilation(nucleus_mask, disk(1))
        distance = ndi.distance_transform_edt(nucleus_mask)
        coords = peak_local_max(
            distance, min_distance=2, labels=nucleus_mask, exclude_border=False
        )

        if len(coords) == 0:
            return label(nucleus_mask)

        mask_centers = np.zeros(distance.shape, dtype=bool)
        mask_centers[tuple(coords.T)] = True
        markers = label(mask_centers)
        labeled_nuclei = watershed(-distance, markers, mask=nucleus_mask)

        return labeled_nuclei

# ============================================================================
# STEP 4: FEATURE EXTRACTORS
# ============================================================================

class NucleusFeatureExtractor:

    @staticmethod
    def extract_nucleus_features(region, intensity_image):
        features = {}
        features['area'] = float(region.area)
        features['perimeter'] = float(region.perimeter)
        features['eccentricity'] = float(region.eccentricity)
        features['solidity'] = float(region.solidity)
        features['extent'] = float(region.extent)
        features['orientation'] = float(region.orientation)
        features['circularity'] = (
            float(4 * np.pi * region.area / (region.perimeter ** 2))
            if region.perimeter > 0 else 0.0
        )
        features['major_axis_length'] = float(region.major_axis_length)
        features['minor_axis_length'] = float(region.minor_axis_length)
        features['aspect_ratio'] = (
            float(region.major_axis_length / region.minor_axis_length)
            if region.minor_axis_length > 0 else 0.0
        )
        bbox = region.bbox
        features['bbox_width'] = float(bbox[3] - bbox[1])
        features['bbox_height'] = float(bbox[2] - bbox[0])
        features['bbox_area'] = float(features['bbox_width'] * features['bbox_height'])

        nucleus_pixels = intensity_image[region.coords[:, 0], region.coords[:, 1]]
        features['intensity_mean'] = float(np.mean(nucleus_pixels))
        features['intensity_std'] = float(np.std(nucleus_pixels))
        features['intensity_min'] = float(np.min(nucleus_pixels))
        features['intensity_max'] = float(np.max(nucleus_pixels))
        features['intensity_range'] = float(np.max(nucleus_pixels) - np.min(nucleus_pixels))
        features['intensity_median'] = float(np.median(nucleus_pixels))
        features['intensity_skewness'] = float(skew(nucleus_pixels))
        features['intensity_kurtosis'] = float(kurtosis(nucleus_pixels))
        return features


class SpatialFeatureExtractor:

    @staticmethod
    def extract_spatial_features(regions):
        default_keys = [
            'nucleus_count', 'total_nucleus_area',
            'nn_distance_mean', 'nn_distance_std', 'nn_distance_min',
            'nn_distance_max', 'nn_distance_median',
            'pairwise_distance_mean', 'pairwise_distance_std',
            'pairwise_distance_min', 'pairwise_distance_max',
            'centroid_x_variance', 'centroid_y_variance',
            'centroid_x_range', 'centroid_y_range',
        ]
        features = {k: 0.0 for k in default_keys}

        if len(regions) == 0:
            return features

        centroids = np.array([region.centroid for region in regions])
        features['nucleus_count'] = float(len(regions))
        features['total_nucleus_area'] = float(sum(r.area for r in regions))

        if len(centroids) > 1:
            distances = cdist(centroids, centroids)
            np.fill_diagonal(distances, np.inf)
            nn_distances = np.min(distances, axis=1)
            features['nn_distance_mean'] = float(np.mean(nn_distances))
            features['nn_distance_std'] = float(np.std(nn_distances))
            features['nn_distance_min'] = float(np.min(nn_distances))
            features['nn_distance_max'] = float(np.max(nn_distances))
            features['nn_distance_median'] = float(np.median(nn_distances))

            upper_triangle = distances[np.triu_indices_from(distances, k=1)]
            finite_vals = upper_triangle[np.isfinite(upper_triangle)]
            if len(finite_vals) > 0:
                features['pairwise_distance_mean'] = float(np.mean(finite_vals))
                features['pairwise_distance_std'] = float(np.std(finite_vals))
                features['pairwise_distance_min'] = float(np.min(finite_vals))
                features['pairwise_distance_max'] = float(np.max(finite_vals))

        if len(centroids) > 2:
            features['centroid_x_variance'] = float(np.var(centroids[:, 1]))
            features['centroid_y_variance'] = float(np.var(centroids[:, 0]))
            features['centroid_x_range'] = float(np.max(centroids[:, 1]) - np.min(centroids[:, 1]))
            features['centroid_y_range'] = float(np.max(centroids[:, 0]) - np.min(centroids[:, 0]))

        return features


class GraphFeatureExtractor:

    @staticmethod
    def build_delaunay_graph(centroids, max_edge_length=100):
        from scipy.spatial import Delaunay
        if len(centroids) < 3:
            return None, None
        try:
            tri = Delaunay(centroids)
            edges = set()
            for simplex in tri.simplices:
                for i in range(3):
                    edge = tuple(sorted([simplex[i], simplex[(i + 1) % 3]]))
                    dist = np.linalg.norm(centroids[edge[0]] - centroids[edge[1]])
                    if dist < max_edge_length:
                        edges.add(edge)
            return tri, list(edges)
        except Exception:
            return None, None

    @staticmethod
    def extract_graph_features(regions):
        default_keys = [
            'num_edges', 'edge_density',
            'degree_mean', 'degree_std', 'degree_max', 'degree_min',
            'edge_length_mean', 'edge_length_std',
            'edge_length_min', 'edge_length_max',
        ]
        features = {k: 0.0 for k in default_keys}

        if len(regions) < 3:
            return features

        centroids = np.array([region.centroid for region in regions])
        tri, edges = GraphFeatureExtractor.build_delaunay_graph(centroids)

        if edges is None or len(edges) == 0:
            return features

        features['num_edges'] = float(len(edges))
        n = len(regions)
        features['edge_density'] = float(len(edges) / (n * (n - 1) / 2))

        degree_count = {}
        for edge in edges:
            degree_count[edge[0]] = degree_count.get(edge[0], 0) + 1
            degree_count[edge[1]] = degree_count.get(edge[1], 0) + 1

        if degree_count:
            degrees = list(degree_count.values())
            features['degree_mean'] = float(np.mean(degrees))
            features['degree_std'] = float(np.std(degrees))
            features['degree_max'] = float(np.max(degrees))
            features['degree_min'] = float(np.min(degrees))

        edge_lengths = [
            np.linalg.norm(centroids[e[0]] - centroids[e[1]]) for e in edges
        ]
        features['edge_length_mean'] = float(np.mean(edge_lengths))
        features['edge_length_std'] = float(np.std(edge_lengths))
        features['edge_length_min'] = float(np.min(edge_lengths))
        features['edge_length_max'] = float(np.max(edge_lengths))

        return features


class ComprehensiveObjectFeatureExtractor:

    def __init__(self):
        self.nucleus_extractor = NucleusFeatureExtractor()
        self.spatial_extractor = SpatialFeatureExtractor()
        self.graph_extractor = GraphFeatureExtractor()

    def extract_all_features(self, rgb_patch):
        all_features = {}

        labeled_nuclei = NucleusSegmenter.segment_nuclei(rgb_patch)
        gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
        regions = regionprops(labeled_nuclei, intensity_image=gray)
        regions = [r for r in regions if r.area >= 8]

        all_features['nucleus_count'] = len(regions)
        all_features['total_nucleus_area'] = float(sum(r.area for r in regions)) if regions else 0.0

        if len(regions) > 0:
            nucleus_features_list = []
            for region in regions:
                try:
                    nuc_feat = self.nucleus_extractor.extract_nucleus_features(region, gray)
                    nucleus_features_list.append(nuc_feat)
                except Exception:
                    continue

            if nucleus_features_list:
                df = pd.DataFrame(nucleus_features_list)
                for col in df.columns:
                    all_features[f'nucleus_{col}_mean'] = float(df[col].mean())
                    all_features[f'nucleus_{col}_std'] = float(df[col].std())
                    all_features[f'nucleus_{col}_min'] = float(df[col].min())
                    all_features[f'nucleus_{col}_max'] = float(df[col].max())
                    all_features[f'nucleus_{col}_median'] = float(df[col].median())

        spatial_features = self.spatial_extractor.extract_spatial_features(regions)
        all_features.update(spatial_features)

        graph_features = self.graph_extractor.extract_graph_features(regions)
        all_features.update(graph_features)

        return all_features

# ============================================================================
# STEP 5: WSI PATCH EXTRACTION
# ============================================================================

def extract_object_features_from_wsi(wsi_path, patch_size=256, target_level=1, tissue_threshold=0.3):
    slide = openslide.OpenSlide(wsi_path)
    tissue_mask, mask_level = build_tissue_mask(slide)
    mask_scale = slide.level_downsamples[mask_level]
    target_downsample = slide.level_downsamples[target_level]

    extractor = ComprehensiveObjectFeatureExtractor()
    features_list = []
    width, height = slide.level_dimensions[target_level]

    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            x_level0 = int(x * target_downsample)
            y_level0 = int(y * target_downsample)
            patch_size_level0 = int(patch_size * target_downsample)

            ratio = tissue_ratio(tissue_mask, x_level0, y_level0, patch_size_level0, mask_scale)

            if ratio < tissue_threshold:
                continue

            patch = slide.read_region((x_level0, y_level0), target_level, (patch_size, patch_size))
            patch = np.array(patch)[:, :, :3]

            try:
                patch_features = extractor.extract_all_features(patch)
                patch_features['patch_x'] = x
                patch_features['patch_y'] = y
                patch_features['tissue_ratio'] = float(ratio)
                features_list.append(patch_features)
            except Exception:
                continue

    slide.close()
    return features_list

# ============================================================================
# STEP 6: SLIDE-LEVEL AGGREGATION
# ============================================================================

def aggregate_patch_features_to_slide_level(patch_features_list, slide_name):
    if not patch_features_list:
        return {}

    df = pd.DataFrame(patch_features_list)
    metadata_cols = ['patch_x', 'patch_y', 'tissue_ratio']
    feature_cols = [col for col in df.columns if col not in metadata_cols]

    slide_features = {'slide_name': slide_name, 'num_patches': len(patch_features_list)}

    for feature in feature_cols:
        feature_values = df[feature].dropna().values
        if len(feature_values) > 0:
            slide_features[f'{feature}_mean'] = float(np.mean(feature_values))
            slide_features[f'{feature}_std'] = float(np.std(feature_values))
            slide_features[f'{feature}_min'] = float(np.min(feature_values))
            slide_features[f'{feature}_max'] = float(np.max(feature_values))
            slide_features[f'{feature}_median'] = float(np.median(feature_values))
            slide_features[f'{feature}_q25'] = float(np.percentile(feature_values, 25))
            slide_features[f'{feature}_q75'] = float(np.percentile(feature_values, 75))
        else:
            for suffix in ['mean', 'std', 'min', 'max', 'median', 'q25', 'q75']:
                slide_features[f'{feature}_{suffix}'] = 0.0

    slide_features['tissue_ratio_mean'] = float(df['tissue_ratio'].mean())
    slide_features['tissue_ratio_std'] = float(df['tissue_ratio'].std())

    return slide_features

# ============================================================================
# STEP 7: PARALLEL SLIDE PROCESSING
# ============================================================================

def process_single_slide(args):
    slide_path, dataset_name, patch_size, target_level, tissue_threshold = args
    slide_name = os.path.basename(slide_path)
    try:
        patch_features = extract_object_features_from_wsi(
            slide_path, patch_size, target_level, tissue_threshold
        )
        if patch_features:
            slide_features = aggregate_patch_features_to_slide_level(
                patch_features, f"{dataset_name}_{slide_name}"
            )
            return {
                'success': True, 'slide_path': slide_path,
                'slide_name': slide_name, 'features': slide_features,
                'num_patches': len(patch_features), 'error': None
            }
        else:
            return {
                'success': False, 'slide_path': slide_path,
                'slide_name': slide_name, 'features': None,
                'num_patches': 0, 'error': 'No tissue patches found'
            }
    except Exception as e:
        return {
            'success': False, 'slide_path': slide_path,
            'slide_name': slide_name, 'features': None,
            'num_patches': 0, 'error': str(e)
        }

# ============================================================================
# STEP 8: MAIN PROCESSING LOOP
# ============================================================================

print("\n" + "="*80)
print("STARTING SLICED BATCH PROCESSING")
print("="*80)

# Build run-specific output folder so each run's CSV is separate
run_label = f"slides_{SLIDE_START_INDEX}_{SLIDE_END_INDEX - 1}"
output_path = f'/kaggle/working/object_features_{dataset_name}_{run_label}/'
os.makedirs(output_path, exist_ok=True)

# Collect and sort all slides so the index is deterministic across runs
all_slides = []
for root, dirs, files in os.walk(dataset_path):
    all_slides.extend([
        os.path.join(root, f) for f in files
        if f.endswith(('.svs', '.tif', '.tiff', '.ndpi'))
    ])
all_slides = sorted(all_slides)  # IMPORTANT: sort so index is consistent every run

print(f"Total slides in dataset: {len(all_slides)}")

# Apply the slice
slides_to_process = all_slides[SLIDE_START_INDEX:SLIDE_END_INDEX]
print(f"This run will process: {len(slides_to_process)} slides "
      f"(index {SLIDE_START_INDEX} → {SLIDE_END_INDEX - 1})")
for s in slides_to_process:
    print(f"  • {os.path.basename(s)}")

all_slides_features = []
total_processed = 0

for batch_start in range(0, len(slides_to_process), BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, len(slides_to_process))
    batch_slides = slides_to_process[batch_start:batch_end]

    print(f"\n{'='*80}")
    print(f"BATCH {batch_start // BATCH_SIZE + 1} — "
          f"slides {SLIDE_START_INDEX + batch_start + 1}–"
          f"{SLIDE_START_INDEX + batch_end} of {len(all_slides)} total")
    print(f"{'='*80}")

    slide_args = [(s, dataset_name, 256, 1, 0.3) for s in batch_slides]

    with ProcessPoolExecutor(max_workers=PARALLEL_WORKERS) as executor:
        futures = {executor.submit(process_single_slide, args): args[0] for args in slide_args}

        for future in as_completed(futures):
            slide_path = futures[future]
            try:
                result = future.result()
                if result['success']:
                    print(f"  ✓ {result['slide_name']}: {result['num_patches']} patches")
                    all_slides_features.append(result['features'])
                    total_processed += 1
                else:
                    print(f"  ✗ {result['slide_name']}: {result['error']}")
            except Exception as e:
                print(f"  ✗ Error with {os.path.basename(slide_path)}: {str(e)}")

    # Save interim CSV after every batch
    if all_slides_features:
        temp_df = pd.DataFrame(all_slides_features)
        temp_df.to_csv(f'{output_path}/slide_object_features_temp.csv', index=False)
        print(f"\n  💾 Batch checkpoint: {total_processed} slides saved so far")

# ============================================================================
# STEP 9: SAVE THIS RUN'S FINAL CSV
# ============================================================================

print("\n" + "="*80)
print("SAVING RUN RESULTS")
print("="*80)

if all_slides_features:
    final_df = pd.DataFrame(all_slides_features)
    print(f"Slides processed this run: {len(final_df)}")
    print(f"Features per slide: {final_df.shape[1]}")

    # Diagnostics
    nucleus_count_cols = [c for c in final_df.columns if 'nucleus_count' in c.lower()]
    for col in nucleus_count_cols:
        zero_count = (final_df[col] == 0).sum()
        print(f"{col}: mean={final_df[col].mean():.2f}, "
              f"zero slides={zero_count}/{len(final_df)} ({100*zero_count/len(final_df):.1f}%)")

    nan_total = final_df.isna().sum().sum()
    print(f"Total NaN values: {nan_total} ({'none ✓' if nan_total == 0 else 'check merge script'})")

    csv_path = f'{output_path}/{dataset_name}_{run_label}_features.csv'
    final_df.to_csv(csv_path, index=False)
    print(f"\n✓ Saved: {csv_path}")
else:
    print("⚠ No slides were successfully processed this run!")

# ============================================================================
# STEP 10: UPLOAD THIS RUN'S OUTPUT TO KAGGLE DATASETS
# ============================================================================

from kaggle import api

dataset_slug = f"object-features-{dataset_name.replace('_', '-')}-{run_label.replace('_', '-')}"
metadata = {
    "title": f"Object Features {dataset_name} {run_label}",
    "id": f"onyekamuoka/{dataset_slug}",
    "licenses": [{"name": "CC0-1.0"}]
}

with open(f'{output_path}/dataset-metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

try:
    api.dataset_create_new(
        folder=output_path, dir_mode='zip', convert_to_csv=False, public=False
    )
    print(f"✓ Created Kaggle dataset: {dataset_slug}")
except Exception:
    api.dataset_create_version(
        folder=output_path,
        version_notes=f"Run {run_label}",
        dir_mode='zip', convert_to_csv=False
    )
    print(f"✓ Updated Kaggle dataset: {dataset_slug}")

print(f"\n✓ https://www.kaggle.com/datasets/onyekamuoka/{dataset_slug}")
print("\n" + "="*80)
print(f"RUN COMPLETE — {run_label}")
print("Change SLIDE_START_INDEX and SLIDE_END_INDEX and run again for next slice.")
print("When all runs done, use bracs_wsi_merge.py to combine all CSVs.")
print("="*80)

# Object Level Take two

# ============================================================================
# KAGGLE NOTEBOOK: BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION (SLICED RUNS)
# ============================================================================
#
# HOW TO USE:
#   This notebook processes a specific slice of slides to stay within
#   Kaggle's 12-hour time limit. Run it multiple times, changing only
#   SLIDE_START_INDEX and SLIDE_END_INDEX each time.
#
#   Recommended splits for UDH (74 slides, 31 already done):
#   ┌─────────┬─────────────────────┬──────────────────┐
#   │  Run    │  Slide Range        │  Change To       │
#   ├─────────┼─────────────────────┼──────────────────┤
#   │  Run A  │  slides 32 – 47    │  START=32 END=48 │
#   │  Run B  │  slides 48 – 63    │  START=48 END=64 │
#   │  Run C  │  slides 64 – 74    │  START=64 END=74 │
#   └─────────┴─────────────────────┴──────────────────┘
#
#   After all runs complete, use bracs_wsi_merge.py to combine the CSVs.
#
# ============================================================================

import os
import numpy as np
import cv2
import kagglehub
import openslide
from skimage.color import rgb2hsv
from skimage.morphology import remove_small_holes, remove_small_objects, disk, dilation
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from scipy.spatial.distance import cdist
from scipy.stats import skew, kurtosis
import json
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# *** CHANGE THESE TWO NUMBERS EACH RUN ***
# ============================================================================

SLIDE_START_INDEX = 0   # first slide to process (0-based index into sorted slide list)
SLIDE_END_INDEX   = 15   # process up to but NOT including this index

# ============================================================================
# CONFIGURATION (leave these unchanged across all runs)
# ============================================================================

PARALLEL_WORKERS = 4
BATCH_SIZE = 8
dataset_to_process = "saadinn/bracs-wsi-group-mt-type-ic-01"

# ============================================================================
# SETUP
# ============================================================================

print("="*80)
print("BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION - SLICED RUN")
print("="*80)
print(f"\nSlide slice: index {SLIDE_START_INDEX} → {SLIDE_END_INDEX - 1}")
print(f"Downloading dataset: {dataset_to_process}")

dataset_path = kagglehub.dataset_download(dataset_to_process)
dataset_name = dataset_to_process.split('/')[-1]

print(f"✓ Downloaded to: {dataset_path}")
print(f"✓ Dataset name:  {dataset_name}")

# ============================================================================
# STEP 2: TISSUE DETECTION
# ============================================================================

def build_tissue_mask(slide):
    mask_level = slide.level_count - 1
    img = np.array(
        slide.read_region((0, 0), mask_level, slide.level_dimensions[mask_level])
    )[:, :, :3]

    hsv = rgb2hsv(img)
    saturation = hsv[:, :, 1]
    value = hsv[:, :, 2]

    mask = (saturation > 0.03) & (value < 0.95)
    mask = remove_small_objects(mask, min_size=300)
    mask = remove_small_holes(mask, area_threshold=300)
    return mask, mask_level


def tissue_ratio(mask, x_level0, y_level0, patch_size_level0, scale):
    xs = int(x_level0 / scale)
    ys = int(y_level0 / scale)
    ps = max(int(patch_size_level0 / scale), 1)
    patch_mask = mask[ys:ys + ps, xs:xs + ps]
    if patch_mask.size == 0:
        return 0.0
    return float(patch_mask.mean())

# ============================================================================
# STEP 3: NUCLEUS SEGMENTATION
# ============================================================================

class NucleusSegmenter:

    @staticmethod
    def extract_hematoxylin_channel(rgb_patch):
        rgb_norm = rgb_patch.astype(np.float32) / 255.0
        rgb_norm = np.clip(rgb_norm, 1e-6, 1.0)
        od = -np.log(rgb_norm)

        HE_matrix = np.array([
            [0.65, 0.70, 0.29],
            [0.07, 0.99, 0.11],
            [0.27, 0.57, 0.78]
        ])

        od_flat = od.reshape(-1, 3)
        stains = np.linalg.lstsq(HE_matrix.T, od_flat.T, rcond=None)[0].T
        hematoxylin = stains[:, 0].reshape(rgb_patch.shape[:2])
        hematoxylin = np.clip(hematoxylin, 0, None)
        hematoxylin = (hematoxylin / (hematoxylin.max() + 1e-6) * 255).astype(np.uint8)
        return hematoxylin

    @staticmethod
    def segment_nuclei(rgb_patch):
        try:
            hematoxylin = NucleusSegmenter.extract_hematoxylin_channel(rgb_patch)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            hematoxylin_eq = clahe.apply(hematoxylin)

            _, nucleus_mask_otsu = cv2.threshold(
                hematoxylin_eq, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
            )

            otsu_ratio = nucleus_mask_otsu.mean() / 255.0
            if otsu_ratio < 0.05 or otsu_ratio > 0.70:
                nucleus_mask = cv2.adaptiveThreshold(
                    hematoxylin_eq, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY,
                    blockSize=31,
                    C=-2
                )
                nucleus_mask = nucleus_mask > 0
            else:
                nucleus_mask = nucleus_mask_otsu > 0

        except Exception:
            gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
            gray_inv = 255 - gray
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            gray_inv_eq = clahe.apply(gray_inv)
            nucleus_mask = cv2.adaptiveThreshold(
                gray_inv_eq, 255,
                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY,
                blockSize=31,
                C=-2
            )
            nucleus_mask = nucleus_mask > 0

        nucleus_mask = remove_small_objects(nucleus_mask, min_size=8)
        nucleus_mask = remove_small_holes(nucleus_mask, area_threshold=8)

        if not nucleus_mask.any():
            return np.zeros(rgb_patch.shape[:2], dtype=int)

        nucleus_mask = dilation(nucleus_mask, disk(1))
        distance = ndi.distance_transform_edt(nucleus_mask)
        coords = peak_local_max(
            distance, min_distance=2, labels=nucleus_mask, exclude_border=False
        )

        if len(coords) == 0:
            return label(nucleus_mask)

        mask_centers = np.zeros(distance.shape, dtype=bool)
        mask_centers[tuple(coords.T)] = True
        markers = label(mask_centers)
        labeled_nuclei = watershed(-distance, markers, mask=nucleus_mask)

        return labeled_nuclei

# ============================================================================
# STEP 4: FEATURE EXTRACTORS
# ============================================================================

class NucleusFeatureExtractor:

    @staticmethod
    def extract_nucleus_features(region, intensity_image):
        features = {}
        features['area'] = float(region.area)
        features['perimeter'] = float(region.perimeter)
        features['eccentricity'] = float(region.eccentricity)
        features['solidity'] = float(region.solidity)
        features['extent'] = float(region.extent)
        features['orientation'] = float(region.orientation)
        features['circularity'] = (
            float(4 * np.pi * region.area / (region.perimeter ** 2))
            if region.perimeter > 0 else 0.0
        )
        features['major_axis_length'] = float(region.major_axis_length)
        features['minor_axis_length'] = float(region.minor_axis_length)
        features['aspect_ratio'] = (
            float(region.major_axis_length / region.minor_axis_length)
            if region.minor_axis_length > 0 else 0.0
        )
        bbox = region.bbox
        features['bbox_width'] = float(bbox[3] - bbox[1])
        features['bbox_height'] = float(bbox[2] - bbox[0])
        features['bbox_area'] = float(features['bbox_width'] * features['bbox_height'])

        nucleus_pixels = intensity_image[region.coords[:, 0], region.coords[:, 1]]
        features['intensity_mean'] = float(np.mean(nucleus_pixels))
        features['intensity_std'] = float(np.std(nucleus_pixels))
        features['intensity_min'] = float(np.min(nucleus_pixels))
        features['intensity_max'] = float(np.max(nucleus_pixels))
        features['intensity_range'] = float(np.max(nucleus_pixels) - np.min(nucleus_pixels))
        features['intensity_median'] = float(np.median(nucleus_pixels))
        features['intensity_skewness'] = float(skew(nucleus_pixels))
        features['intensity_kurtosis'] = float(kurtosis(nucleus_pixels))
        return features


class SpatialFeatureExtractor:

    @staticmethod
    def extract_spatial_features(regions):
        default_keys = [
            'nucleus_count', 'total_nucleus_area',
            'nn_distance_mean', 'nn_distance_std', 'nn_distance_min',
            'nn_distance_max', 'nn_distance_median',
            'pairwise_distance_mean', 'pairwise_distance_std',
            'pairwise_distance_min', 'pairwise_distance_max',
            'centroid_x_variance', 'centroid_y_variance',
            'centroid_x_range', 'centroid_y_range',
        ]
        features = {k: 0.0 for k in default_keys}

        if len(regions) == 0:
            return features

        centroids = np.array([region.centroid for region in regions])
        features['nucleus_count'] = float(len(regions))
        features['total_nucleus_area'] = float(sum(r.area for r in regions))

        if len(centroids) > 1:
            distances = cdist(centroids, centroids)
            np.fill_diagonal(distances, np.inf)
            nn_distances = np.min(distances, axis=1)
            features['nn_distance_mean'] = float(np.mean(nn_distances))
            features['nn_distance_std'] = float(np.std(nn_distances))
            features['nn_distance_min'] = float(np.min(nn_distances))
            features['nn_distance_max'] = float(np.max(nn_distances))
            features['nn_distance_median'] = float(np.median(nn_distances))

            upper_triangle = distances[np.triu_indices_from(distances, k=1)]
            finite_vals = upper_triangle[np.isfinite(upper_triangle)]
            if len(finite_vals) > 0:
                features['pairwise_distance_mean'] = float(np.mean(finite_vals))
                features['pairwise_distance_std'] = float(np.std(finite_vals))
                features['pairwise_distance_min'] = float(np.min(finite_vals))
                features['pairwise_distance_max'] = float(np.max(finite_vals))

        if len(centroids) > 2:
            features['centroid_x_variance'] = float(np.var(centroids[:, 1]))
            features['centroid_y_variance'] = float(np.var(centroids[:, 0]))
            features['centroid_x_range'] = float(np.max(centroids[:, 1]) - np.min(centroids[:, 1]))
            features['centroid_y_range'] = float(np.max(centroids[:, 0]) - np.min(centroids[:, 0]))

        return features


class GraphFeatureExtractor:

    @staticmethod
    def build_delaunay_graph(centroids, max_edge_length=100):
        from scipy.spatial import Delaunay
        if len(centroids) < 3:
            return None, None
        try:
            tri = Delaunay(centroids)
            edges = set()
            for simplex in tri.simplices:
                for i in range(3):
                    edge = tuple(sorted([simplex[i], simplex[(i + 1) % 3]]))
                    dist = np.linalg.norm(centroids[edge[0]] - centroids[edge[1]])
                    if dist < max_edge_length:
                        edges.add(edge)
            return tri, list(edges)
        except Exception:
            return None, None

    @staticmethod
    def extract_graph_features(regions):
        default_keys = [
            'num_edges', 'edge_density',
            'degree_mean', 'degree_std', 'degree_max', 'degree_min',
            'edge_length_mean', 'edge_length_std',
            'edge_length_min', 'edge_length_max',
        ]
        features = {k: 0.0 for k in default_keys}

        if len(regions) < 3:
            return features

        centroids = np.array([region.centroid for region in regions])
        tri, edges = GraphFeatureExtractor.build_delaunay_graph(centroids)

        if edges is None or len(edges) == 0:
            return features

        features['num_edges'] = float(len(edges))
        n = len(regions)
        features['edge_density'] = float(len(edges) / (n * (n - 1) / 2))

        degree_count = {}
        for edge in edges:
            degree_count[edge[0]] = degree_count.get(edge[0], 0) + 1
            degree_count[edge[1]] = degree_count.get(edge[1], 0) + 1

        if degree_count:
            degrees = list(degree_count.values())
            features['degree_mean'] = float(np.mean(degrees))
            features['degree_std'] = float(np.std(degrees))
            features['degree_max'] = float(np.max(degrees))
            features['degree_min'] = float(np.min(degrees))

        edge_lengths = [
            np.linalg.norm(centroids[e[0]] - centroids[e[1]]) for e in edges
        ]
        features['edge_length_mean'] = float(np.mean(edge_lengths))
        features['edge_length_std'] = float(np.std(edge_lengths))
        features['edge_length_min'] = float(np.min(edge_lengths))
        features['edge_length_max'] = float(np.max(edge_lengths))

        return features


class ComprehensiveObjectFeatureExtractor:

    def __init__(self):
        self.nucleus_extractor = NucleusFeatureExtractor()
        self.spatial_extractor = SpatialFeatureExtractor()
        self.graph_extractor = GraphFeatureExtractor()

    def extract_all_features(self, rgb_patch):
        all_features = {}

        labeled_nuclei = NucleusSegmenter.segment_nuclei(rgb_patch)
        gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
        regions = regionprops(labeled_nuclei, intensity_image=gray)
        regions = [r for r in regions if r.area >= 8]

        all_features['nucleus_count'] = len(regions)
        all_features['total_nucleus_area'] = float(sum(r.area for r in regions)) if regions else 0.0

        if len(regions) > 0:
            nucleus_features_list = []
            for region in regions:
                try:
                    nuc_feat = self.nucleus_extractor.extract_nucleus_features(region, gray)
                    nucleus_features_list.append(nuc_feat)
                except Exception:
                    continue

            if nucleus_features_list:
                df = pd.DataFrame(nucleus_features_list)
                for col in df.columns:
                    all_features[f'nucleus_{col}_mean'] = float(df[col].mean())
                    all_features[f'nucleus_{col}_std'] = float(df[col].std())
                    all_features[f'nucleus_{col}_min'] = float(df[col].min())
                    all_features[f'nucleus_{col}_max'] = float(df[col].max())
                    all_features[f'nucleus_{col}_median'] = float(df[col].median())

        spatial_features = self.spatial_extractor.extract_spatial_features(regions)
        all_features.update(spatial_features)

        graph_features = self.graph_extractor.extract_graph_features(regions)
        all_features.update(graph_features)

        return all_features

# ============================================================================
# STEP 5: WSI PATCH EXTRACTION
# ============================================================================

def extract_object_features_from_wsi(wsi_path, patch_size=256, target_level=1, tissue_threshold=0.3):
    slide = openslide.OpenSlide(wsi_path)
    tissue_mask, mask_level = build_tissue_mask(slide)
    mask_scale = slide.level_downsamples[mask_level]
    target_downsample = slide.level_downsamples[target_level]

    extractor = ComprehensiveObjectFeatureExtractor()
    features_list = []
    width, height = slide.level_dimensions[target_level]

    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            x_level0 = int(x * target_downsample)
            y_level0 = int(y * target_downsample)
            patch_size_level0 = int(patch_size * target_downsample)

            ratio = tissue_ratio(tissue_mask, x_level0, y_level0, patch_size_level0, mask_scale)

            if ratio < tissue_threshold:
                continue

            patch = slide.read_region((x_level0, y_level0), target_level, (patch_size, patch_size))
            patch = np.array(patch)[:, :, :3]

            try:
                patch_features = extractor.extract_all_features(patch)
                patch_features['patch_x'] = x
                patch_features['patch_y'] = y
                patch_features['tissue_ratio'] = float(ratio)
                features_list.append(patch_features)
            except Exception:
                continue

    slide.close()
    return features_list

# ============================================================================
# STEP 6: SLIDE-LEVEL AGGREGATION
# ============================================================================

def aggregate_patch_features_to_slide_level(patch_features_list, slide_name):
    if not patch_features_list:
        return {}

    df = pd.DataFrame(patch_features_list)
    metadata_cols = ['patch_x', 'patch_y', 'tissue_ratio']
    feature_cols = [col for col in df.columns if col not in metadata_cols]

    slide_features = {'slide_name': slide_name, 'num_patches': len(patch_features_list)}

    for feature in feature_cols:
        feature_values = df[feature].dropna().values
        if len(feature_values) > 0:
            slide_features[f'{feature}_mean'] = float(np.mean(feature_values))
            slide_features[f'{feature}_std'] = float(np.std(feature_values))
            slide_features[f'{feature}_min'] = float(np.min(feature_values))
            slide_features[f'{feature}_max'] = float(np.max(feature_values))
            slide_features[f'{feature}_median'] = float(np.median(feature_values))
            slide_features[f'{feature}_q25'] = float(np.percentile(feature_values, 25))
            slide_features[f'{feature}_q75'] = float(np.percentile(feature_values, 75))
        else:
            for suffix in ['mean', 'std', 'min', 'max', 'median', 'q25', 'q75']:
                slide_features[f'{feature}_{suffix}'] = 0.0

    slide_features['tissue_ratio_mean'] = float(df['tissue_ratio'].mean())
    slide_features['tissue_ratio_std'] = float(df['tissue_ratio'].std())

    return slide_features

# ============================================================================
# STEP 7: PARALLEL SLIDE PROCESSING
# ============================================================================

def process_single_slide(args):
    slide_path, dataset_name, patch_size, target_level, tissue_threshold = args
    slide_name = os.path.basename(slide_path)
    try:
        patch_features = extract_object_features_from_wsi(
            slide_path, patch_size, target_level, tissue_threshold
        )
        if patch_features:
            slide_features = aggregate_patch_features_to_slide_level(
                patch_features, f"{dataset_name}_{slide_name}"
            )
            return {
                'success': True, 'slide_path': slide_path,
                'slide_name': slide_name, 'features': slide_features,
                'num_patches': len(patch_features), 'error': None
            }
        else:
            return {
                'success': False, 'slide_path': slide_path,
                'slide_name': slide_name, 'features': None,
                'num_patches': 0, 'error': 'No tissue patches found'
            }
    except Exception as e:
        return {
            'success': False, 'slide_path': slide_path,
            'slide_name': slide_name, 'features': None,
            'num_patches': 0, 'error': str(e)
        }

# ============================================================================
# STEP 8: MAIN PROCESSING LOOP
# ============================================================================

print("\n" + "="*80)
print("STARTING SLICED BATCH PROCESSING")
print("="*80)

# Build run-specific output folder so each run's CSV is separate
run_label = f"slides_{SLIDE_START_INDEX}_{SLIDE_END_INDEX - 1}"
output_path = f'/kaggle/working/object_features_{dataset_name}_{run_label}/'
os.makedirs(output_path, exist_ok=True)

# Collect and sort all slides so the index is deterministic across runs
all_slides = []
for root, dirs, files in os.walk(dataset_path):
    all_slides.extend([
        os.path.join(root, f) for f in files
        if f.endswith(('.svs', '.tif', '.tiff', '.ndpi'))
    ])
all_slides = sorted(all_slides)  # IMPORTANT: sort so index is consistent every run

print(f"Total slides in dataset: {len(all_slides)}")

# Apply the slice
slides_to_process = all_slides[SLIDE_START_INDEX:SLIDE_END_INDEX]
print(f"This run will process: {len(slides_to_process)} slides "
      f"(index {SLIDE_START_INDEX} → {SLIDE_END_INDEX - 1})")
for s in slides_to_process:
    print(f"  • {os.path.basename(s)}")

all_slides_features = []
total_processed = 0

for batch_start in range(0, len(slides_to_process), BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, len(slides_to_process))
    batch_slides = slides_to_process[batch_start:batch_end]

    print(f"\n{'='*80}")
    print(f"BATCH {batch_start // BATCH_SIZE + 1} — "
          f"slides {SLIDE_START_INDEX + batch_start + 1}–"
          f"{SLIDE_START_INDEX + batch_end} of {len(all_slides)} total")
    print(f"{'='*80}")

    slide_args = [(s, dataset_name, 256, 1, 0.3) for s in batch_slides]

    with ProcessPoolExecutor(max_workers=PARALLEL_WORKERS) as executor:
        futures = {executor.submit(process_single_slide, args): args[0] for args in slide_args}

        for future in as_completed(futures):
            slide_path = futures[future]
            try:
                result = future.result()
                if result['success']:
                    print(f"  ✓ {result['slide_name']}: {result['num_patches']} patches")
                    all_slides_features.append(result['features'])
                    total_processed += 1
                else:
                    print(f"  ✗ {result['slide_name']}: {result['error']}")
            except Exception as e:
                print(f"  ✗ Error with {os.path.basename(slide_path)}: {str(e)}")

    # Save interim CSV after every batch
    if all_slides_features:
        temp_df = pd.DataFrame(all_slides_features)
        temp_df.to_csv(f'{output_path}/slide_object_features_temp.csv', index=False)
        print(f"\n  💾 Batch checkpoint: {total_processed} slides saved so far")

# ============================================================================
# STEP 9: SAVE THIS RUN'S FINAL CSV
# ============================================================================

print("\n" + "="*80)
print("SAVING RUN RESULTS")
print("="*80)

if all_slides_features:
    final_df = pd.DataFrame(all_slides_features)
    print(f"Slides processed this run: {len(final_df)}")
    print(f"Features per slide: {final_df.shape[1]}")

    # Diagnostics
    nucleus_count_cols = [c for c in final_df.columns if 'nucleus_count' in c.lower()]
    for col in nucleus_count_cols:
        zero_count = (final_df[col] == 0).sum()
        print(f"{col}: mean={final_df[col].mean():.2f}, "
              f"zero slides={zero_count}/{len(final_df)} ({100*zero_count/len(final_df):.1f}%)")

    nan_total = final_df.isna().sum().sum()
    print(f"Total NaN values: {nan_total} ({'none ✓' if nan_total == 0 else 'check merge script'})")

    csv_path = f'{output_path}/{dataset_name}_{run_label}_features.csv'
    final_df.to_csv(csv_path, index=False)
    print(f"\n✓ Saved: {csv_path}")
else:
    print("⚠ No slides were successfully processed this run!")

# ============================================================================
# STEP 10: UPLOAD THIS RUN'S OUTPUT TO KAGGLE DATASETS
# ============================================================================

from kaggle import api

dataset_slug = f"object-features-{dataset_name.replace('_', '-')}-{run_label.replace('_', '-')}"
metadata = {
    "title": f"Object Features {dataset_name} {run_label}",
    "id": f"onyekamuoka/{dataset_slug}",
    "licenses": [{"name": "CC0-1.0"}]
}

with open(f'{output_path}/dataset-metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

try:
    api.dataset_create_new(
        folder=output_path, dir_mode='zip', convert_to_csv=False, public=False
    )
    print(f"✓ Created Kaggle dataset: {dataset_slug}")
except Exception:
    api.dataset_create_version(
        folder=output_path,
        version_notes=f"Run {run_label}",
        dir_mode='zip', convert_to_csv=False
    )
    print(f"✓ Updated Kaggle dataset: {dataset_slug}")

print(f"\n✓ https://www.kaggle.com/datasets/onyekamuoka/{dataset_slug}")
print("\n" + "="*80)
print(f"RUN COMPLETE — {run_label}")
print("Change SLIDE_START_INDEX and SLIDE_END_INDEX and run again for next slice.")
print("When all runs done, use bracs_wsi_merge.py to combine all CSVs.")
print("="*80)

# ============================================================================
# KAGGLE NOTEBOOK: BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION (SLICED RUNS)
# ============================================================================
#
# HOW TO USE:
#   This notebook processes a specific slice of slides to stay within
#   Kaggle's 12-hour time limit. Run it multiple times, changing only
#   SLIDE_START_INDEX and SLIDE_END_INDEX each time.
#
#   Recommended splits for UDH (74 slides, 31 already done):
#   ┌─────────┬─────────────────────┬──────────────────┐
#   │  Run    │  Slide Range        │  Change To       │
#   ├─────────┼─────────────────────┼──────────────────┤
#   │  Run A  │  slides 32 – 47    │  START=32 END=48 │
#   │  Run B  │  slides 48 – 63    │  START=48 END=64 │
#   │  Run C  │  slides 64 – 74    │  START=64 END=74 │
#   └─────────┴─────────────────────┴──────────────────┘
#
#   After all runs complete, use bracs_wsi_merge.py to combine the CSVs.
#
# ============================================================================

import os
import numpy as np
import cv2
import kagglehub
import openslide
from skimage.color import rgb2hsv
from skimage.morphology import remove_small_holes, remove_small_objects, disk, dilation
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from scipy.spatial.distance import cdist
from scipy.stats import skew, kurtosis
import json
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# *** CHANGE THESE TWO NUMBERS EACH RUN ***
# ============================================================================

SLIDE_START_INDEX = 15   # first slide to process (0-based index into sorted slide list)
SLIDE_END_INDEX   = 30   # process up to but NOT including this index

# ============================================================================
# CONFIGURATION (leave these unchanged across all runs)
# ============================================================================

PARALLEL_WORKERS = 4
BATCH_SIZE = 8
dataset_to_process = "saadinn/bracs-wsi-group-mt-type-ic-01"

# ============================================================================
# SETUP
# ============================================================================

print("="*80)
print("BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION - SLICED RUN")
print("="*80)
print(f"\nSlide slice: index {SLIDE_START_INDEX} → {SLIDE_END_INDEX - 1}")
print(f"Downloading dataset: {dataset_to_process}")

dataset_path = kagglehub.dataset_download(dataset_to_process)
dataset_name = dataset_to_process.split('/')[-1]

print(f"✓ Downloaded to: {dataset_path}")
print(f"✓ Dataset name:  {dataset_name}")

# ============================================================================
# STEP 2: TISSUE DETECTION
# ============================================================================

def build_tissue_mask(slide):
    mask_level = slide.level_count - 1
    img = np.array(
        slide.read_region((0, 0), mask_level, slide.level_dimensions[mask_level])
    )[:, :, :3]

    hsv = rgb2hsv(img)
    saturation = hsv[:, :, 1]
    value = hsv[:, :, 2]

    mask = (saturation > 0.03) & (value < 0.95)
    mask = remove_small_objects(mask, min_size=300)
    mask = remove_small_holes(mask, area_threshold=300)
    return mask, mask_level


def tissue_ratio(mask, x_level0, y_level0, patch_size_level0, scale):
    xs = int(x_level0 / scale)
    ys = int(y_level0 / scale)
    ps = max(int(patch_size_level0 / scale), 1)
    patch_mask = mask[ys:ys + ps, xs:xs + ps]
    if patch_mask.size == 0:
        return 0.0
    return float(patch_mask.mean())

# ============================================================================
# STEP 3: NUCLEUS SEGMENTATION
# ============================================================================

class NucleusSegmenter:

    @staticmethod
    def extract_hematoxylin_channel(rgb_patch):
        rgb_norm = rgb_patch.astype(np.float32) / 255.0
        rgb_norm = np.clip(rgb_norm, 1e-6, 1.0)
        od = -np.log(rgb_norm)

        HE_matrix = np.array([
            [0.65, 0.70, 0.29],
            [0.07, 0.99, 0.11],
            [0.27, 0.57, 0.78]
        ])

        od_flat = od.reshape(-1, 3)
        stains = np.linalg.lstsq(HE_matrix.T, od_flat.T, rcond=None)[0].T
        hematoxylin = stains[:, 0].reshape(rgb_patch.shape[:2])
        hematoxylin = np.clip(hematoxylin, 0, None)
        hematoxylin = (hematoxylin / (hematoxylin.max() + 1e-6) * 255).astype(np.uint8)
        return hematoxylin

    @staticmethod
    def segment_nuclei(rgb_patch):
        try:
            hematoxylin = NucleusSegmenter.extract_hematoxylin_channel(rgb_patch)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            hematoxylin_eq = clahe.apply(hematoxylin)

            _, nucleus_mask_otsu = cv2.threshold(
                hematoxylin_eq, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
            )

            otsu_ratio = nucleus_mask_otsu.mean() / 255.0
            if otsu_ratio < 0.05 or otsu_ratio > 0.70:
                nucleus_mask = cv2.adaptiveThreshold(
                    hematoxylin_eq, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY,
                    blockSize=31,
                    C=-2
                )
                nucleus_mask = nucleus_mask > 0
            else:
                nucleus_mask = nucleus_mask_otsu > 0

        except Exception:
            gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
            gray_inv = 255 - gray
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            gray_inv_eq = clahe.apply(gray_inv)
            nucleus_mask = cv2.adaptiveThreshold(
                gray_inv_eq, 255,
                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY,
                blockSize=31,
                C=-2
            )
            nucleus_mask = nucleus_mask > 0

        nucleus_mask = remove_small_objects(nucleus_mask, min_size=8)
        nucleus_mask = remove_small_holes(nucleus_mask, area_threshold=8)

        if not nucleus_mask.any():
            return np.zeros(rgb_patch.shape[:2], dtype=int)

        nucleus_mask = dilation(nucleus_mask, disk(1))
        distance = ndi.distance_transform_edt(nucleus_mask)
        coords = peak_local_max(
            distance, min_distance=2, labels=nucleus_mask, exclude_border=False
        )

        if len(coords) == 0:
            return label(nucleus_mask)

        mask_centers = np.zeros(distance.shape, dtype=bool)
        mask_centers[tuple(coords.T)] = True
        markers = label(mask_centers)
        labeled_nuclei = watershed(-distance, markers, mask=nucleus_mask)

        return labeled_nuclei

# ============================================================================
# STEP 4: FEATURE EXTRACTORS
# ============================================================================

class NucleusFeatureExtractor:

    @staticmethod
    def extract_nucleus_features(region, intensity_image):
        features = {}
        features['area'] = float(region.area)
        features['perimeter'] = float(region.perimeter)
        features['eccentricity'] = float(region.eccentricity)
        features['solidity'] = float(region.solidity)
        features['extent'] = float(region.extent)
        features['orientation'] = float(region.orientation)
        features['circularity'] = (
            float(4 * np.pi * region.area / (region.perimeter ** 2))
            if region.perimeter > 0 else 0.0
        )
        features['major_axis_length'] = float(region.major_axis_length)
        features['minor_axis_length'] = float(region.minor_axis_length)
        features['aspect_ratio'] = (
            float(region.major_axis_length / region.minor_axis_length)
            if region.minor_axis_length > 0 else 0.0
        )
        bbox = region.bbox
        features['bbox_width'] = float(bbox[3] - bbox[1])
        features['bbox_height'] = float(bbox[2] - bbox[0])
        features['bbox_area'] = float(features['bbox_width'] * features['bbox_height'])

        nucleus_pixels = intensity_image[region.coords[:, 0], region.coords[:, 1]]
        features['intensity_mean'] = float(np.mean(nucleus_pixels))
        features['intensity_std'] = float(np.std(nucleus_pixels))
        features['intensity_min'] = float(np.min(nucleus_pixels))
        features['intensity_max'] = float(np.max(nucleus_pixels))
        features['intensity_range'] = float(np.max(nucleus_pixels) - np.min(nucleus_pixels))
        features['intensity_median'] = float(np.median(nucleus_pixels))
        features['intensity_skewness'] = float(skew(nucleus_pixels))
        features['intensity_kurtosis'] = float(kurtosis(nucleus_pixels))
        return features


class SpatialFeatureExtractor:

    @staticmethod
    def extract_spatial_features(regions):
        default_keys = [
            'nucleus_count', 'total_nucleus_area',
            'nn_distance_mean', 'nn_distance_std', 'nn_distance_min',
            'nn_distance_max', 'nn_distance_median',
            'pairwise_distance_mean', 'pairwise_distance_std',
            'pairwise_distance_min', 'pairwise_distance_max',
            'centroid_x_variance', 'centroid_y_variance',
            'centroid_x_range', 'centroid_y_range',
        ]
        features = {k: 0.0 for k in default_keys}

        if len(regions) == 0:
            return features

        centroids = np.array([region.centroid for region in regions])
        features['nucleus_count'] = float(len(regions))
        features['total_nucleus_area'] = float(sum(r.area for r in regions))

        if len(centroids) > 1:
            distances = cdist(centroids, centroids)
            np.fill_diagonal(distances, np.inf)
            nn_distances = np.min(distances, axis=1)
            features['nn_distance_mean'] = float(np.mean(nn_distances))
            features['nn_distance_std'] = float(np.std(nn_distances))
            features['nn_distance_min'] = float(np.min(nn_distances))
            features['nn_distance_max'] = float(np.max(nn_distances))
            features['nn_distance_median'] = float(np.median(nn_distances))

            upper_triangle = distances[np.triu_indices_from(distances, k=1)]
            finite_vals = upper_triangle[np.isfinite(upper_triangle)]
            if len(finite_vals) > 0:
                features['pairwise_distance_mean'] = float(np.mean(finite_vals))
                features['pairwise_distance_std'] = float(np.std(finite_vals))
                features['pairwise_distance_min'] = float(np.min(finite_vals))
                features['pairwise_distance_max'] = float(np.max(finite_vals))

        if len(centroids) > 2:
            features['centroid_x_variance'] = float(np.var(centroids[:, 1]))
            features['centroid_y_variance'] = float(np.var(centroids[:, 0]))
            features['centroid_x_range'] = float(np.max(centroids[:, 1]) - np.min(centroids[:, 1]))
            features['centroid_y_range'] = float(np.max(centroids[:, 0]) - np.min(centroids[:, 0]))

        return features


class GraphFeatureExtractor:

    @staticmethod
    def build_delaunay_graph(centroids, max_edge_length=100):
        from scipy.spatial import Delaunay
        if len(centroids) < 3:
            return None, None
        try:
            tri = Delaunay(centroids)
            edges = set()
            for simplex in tri.simplices:
                for i in range(3):
                    edge = tuple(sorted([simplex[i], simplex[(i + 1) % 3]]))
                    dist = np.linalg.norm(centroids[edge[0]] - centroids[edge[1]])
                    if dist < max_edge_length:
                        edges.add(edge)
            return tri, list(edges)
        except Exception:
            return None, None

    @staticmethod
    def extract_graph_features(regions):
        default_keys = [
            'num_edges', 'edge_density',
            'degree_mean', 'degree_std', 'degree_max', 'degree_min',
            'edge_length_mean', 'edge_length_std',
            'edge_length_min', 'edge_length_max',
        ]
        features = {k: 0.0 for k in default_keys}

        if len(regions) < 3:
            return features

        centroids = np.array([region.centroid for region in regions])
        tri, edges = GraphFeatureExtractor.build_delaunay_graph(centroids)

        if edges is None or len(edges) == 0:
            return features

        features['num_edges'] = float(len(edges))
        n = len(regions)
        features['edge_density'] = float(len(edges) / (n * (n - 1) / 2))

        degree_count = {}
        for edge in edges:
            degree_count[edge[0]] = degree_count.get(edge[0], 0) + 1
            degree_count[edge[1]] = degree_count.get(edge[1], 0) + 1

        if degree_count:
            degrees = list(degree_count.values())
            features['degree_mean'] = float(np.mean(degrees))
            features['degree_std'] = float(np.std(degrees))
            features['degree_max'] = float(np.max(degrees))
            features['degree_min'] = float(np.min(degrees))

        edge_lengths = [
            np.linalg.norm(centroids[e[0]] - centroids[e[1]]) for e in edges
        ]
        features['edge_length_mean'] = float(np.mean(edge_lengths))
        features['edge_length_std'] = float(np.std(edge_lengths))
        features['edge_length_min'] = float(np.min(edge_lengths))
        features['edge_length_max'] = float(np.max(edge_lengths))

        return features


class ComprehensiveObjectFeatureExtractor:

    def __init__(self):
        self.nucleus_extractor = NucleusFeatureExtractor()
        self.spatial_extractor = SpatialFeatureExtractor()
        self.graph_extractor = GraphFeatureExtractor()

    def extract_all_features(self, rgb_patch):
        all_features = {}

        labeled_nuclei = NucleusSegmenter.segment_nuclei(rgb_patch)
        gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
        regions = regionprops(labeled_nuclei, intensity_image=gray)
        regions = [r for r in regions if r.area >= 8]

        all_features['nucleus_count'] = len(regions)
        all_features['total_nucleus_area'] = float(sum(r.area for r in regions)) if regions else 0.0

        if len(regions) > 0:
            nucleus_features_list = []
            for region in regions:
                try:
                    nuc_feat = self.nucleus_extractor.extract_nucleus_features(region, gray)
                    nucleus_features_list.append(nuc_feat)
                except Exception:
                    continue

            if nucleus_features_list:
                df = pd.DataFrame(nucleus_features_list)
                for col in df.columns:
                    all_features[f'nucleus_{col}_mean'] = float(df[col].mean())
                    all_features[f'nucleus_{col}_std'] = float(df[col].std())
                    all_features[f'nucleus_{col}_min'] = float(df[col].min())
                    all_features[f'nucleus_{col}_max'] = float(df[col].max())
                    all_features[f'nucleus_{col}_median'] = float(df[col].median())

        spatial_features = self.spatial_extractor.extract_spatial_features(regions)
        all_features.update(spatial_features)

        graph_features = self.graph_extractor.extract_graph_features(regions)
        all_features.update(graph_features)

        return all_features

# ============================================================================
# STEP 5: WSI PATCH EXTRACTION
# ============================================================================

def extract_object_features_from_wsi(wsi_path, patch_size=256, target_level=1, tissue_threshold=0.3):
    slide = openslide.OpenSlide(wsi_path)
    tissue_mask, mask_level = build_tissue_mask(slide)
    mask_scale = slide.level_downsamples[mask_level]
    target_downsample = slide.level_downsamples[target_level]

    extractor = ComprehensiveObjectFeatureExtractor()
    features_list = []
    width, height = slide.level_dimensions[target_level]

    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            x_level0 = int(x * target_downsample)
            y_level0 = int(y * target_downsample)
            patch_size_level0 = int(patch_size * target_downsample)

            ratio = tissue_ratio(tissue_mask, x_level0, y_level0, patch_size_level0, mask_scale)

            if ratio < tissue_threshold:
                continue

            patch = slide.read_region((x_level0, y_level0), target_level, (patch_size, patch_size))
            patch = np.array(patch)[:, :, :3]

            try:
                patch_features = extractor.extract_all_features(patch)
                patch_features['patch_x'] = x
                patch_features['patch_y'] = y
                patch_features['tissue_ratio'] = float(ratio)
                features_list.append(patch_features)
            except Exception:
                continue

    slide.close()
    return features_list

# ============================================================================
# STEP 6: SLIDE-LEVEL AGGREGATION
# ============================================================================

def aggregate_patch_features_to_slide_level(patch_features_list, slide_name):
    if not patch_features_list:
        return {}

    df = pd.DataFrame(patch_features_list)
    metadata_cols = ['patch_x', 'patch_y', 'tissue_ratio']
    feature_cols = [col for col in df.columns if col not in metadata_cols]

    slide_features = {'slide_name': slide_name, 'num_patches': len(patch_features_list)}

    for feature in feature_cols:
        feature_values = df[feature].dropna().values
        if len(feature_values) > 0:
            slide_features[f'{feature}_mean'] = float(np.mean(feature_values))
            slide_features[f'{feature}_std'] = float(np.std(feature_values))
            slide_features[f'{feature}_min'] = float(np.min(feature_values))
            slide_features[f'{feature}_max'] = float(np.max(feature_values))
            slide_features[f'{feature}_median'] = float(np.median(feature_values))
            slide_features[f'{feature}_q25'] = float(np.percentile(feature_values, 25))
            slide_features[f'{feature}_q75'] = float(np.percentile(feature_values, 75))
        else:
            for suffix in ['mean', 'std', 'min', 'max', 'median', 'q25', 'q75']:
                slide_features[f'{feature}_{suffix}'] = 0.0

    slide_features['tissue_ratio_mean'] = float(df['tissue_ratio'].mean())
    slide_features['tissue_ratio_std'] = float(df['tissue_ratio'].std())

    return slide_features

# ============================================================================
# STEP 7: PARALLEL SLIDE PROCESSING
# ============================================================================

def process_single_slide(args):
    slide_path, dataset_name, patch_size, target_level, tissue_threshold = args
    slide_name = os.path.basename(slide_path)
    try:
        patch_features = extract_object_features_from_wsi(
            slide_path, patch_size, target_level, tissue_threshold
        )
        if patch_features:
            slide_features = aggregate_patch_features_to_slide_level(
                patch_features, f"{dataset_name}_{slide_name}"
            )
            return {
                'success': True, 'slide_path': slide_path,
                'slide_name': slide_name, 'features': slide_features,
                'num_patches': len(patch_features), 'error': None
            }
        else:
            return {
                'success': False, 'slide_path': slide_path,
                'slide_name': slide_name, 'features': None,
                'num_patches': 0, 'error': 'No tissue patches found'
            }
    except Exception as e:
        return {
            'success': False, 'slide_path': slide_path,
            'slide_name': slide_name, 'features': None,
            'num_patches': 0, 'error': str(e)
        }

# ============================================================================
# STEP 8: MAIN PROCESSING LOOP
# ============================================================================

print("\n" + "="*80)
print("STARTING SLICED BATCH PROCESSING")
print("="*80)

# Build run-specific output folder so each run's CSV is separate
run_label = f"slides_{SLIDE_START_INDEX}_{SLIDE_END_INDEX - 1}"
output_path = f'/kaggle/working/object_features_{dataset_name}_{run_label}/'
os.makedirs(output_path, exist_ok=True)

# Collect and sort all slides so the index is deterministic across runs
all_slides = []
for root, dirs, files in os.walk(dataset_path):
    all_slides.extend([
        os.path.join(root, f) for f in files
        if f.endswith(('.svs', '.tif', '.tiff', '.ndpi'))
    ])
all_slides = sorted(all_slides)  # IMPORTANT: sort so index is consistent every run

print(f"Total slides in dataset: {len(all_slides)}")

# Apply the slice
slides_to_process = all_slides[SLIDE_START_INDEX:SLIDE_END_INDEX]
print(f"This run will process: {len(slides_to_process)} slides "
      f"(index {SLIDE_START_INDEX} → {SLIDE_END_INDEX - 1})")
for s in slides_to_process:
    print(f"  • {os.path.basename(s)}")

all_slides_features = []
total_processed = 0

for batch_start in range(0, len(slides_to_process), BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, len(slides_to_process))
    batch_slides = slides_to_process[batch_start:batch_end]

    print(f"\n{'='*80}")
    print(f"BATCH {batch_start // BATCH_SIZE + 1} — "
          f"slides {SLIDE_START_INDEX + batch_start + 1}–"
          f"{SLIDE_START_INDEX + batch_end} of {len(all_slides)} total")
    print(f"{'='*80}")

    slide_args = [(s, dataset_name, 256, 1, 0.3) for s in batch_slides]

    with ProcessPoolExecutor(max_workers=PARALLEL_WORKERS) as executor:
        futures = {executor.submit(process_single_slide, args): args[0] for args in slide_args}

        for future in as_completed(futures):
            slide_path = futures[future]
            try:
                result = future.result()
                if result['success']:
                    print(f"  ✓ {result['slide_name']}: {result['num_patches']} patches")
                    all_slides_features.append(result['features'])
                    total_processed += 1
                else:
                    print(f"  ✗ {result['slide_name']}: {result['error']}")
            except Exception as e:
                print(f"  ✗ Error with {os.path.basename(slide_path)}: {str(e)}")

    # Save interim CSV after every batch
    if all_slides_features:
        temp_df = pd.DataFrame(all_slides_features)
        temp_df.to_csv(f'{output_path}/slide_object_features_temp.csv', index=False)
        print(f"\n  💾 Batch checkpoint: {total_processed} slides saved so far")

# ============================================================================
# STEP 9: SAVE THIS RUN'S FINAL CSV
# ============================================================================

print("\n" + "="*80)
print("SAVING RUN RESULTS")
print("="*80)

if all_slides_features:
    final_df = pd.DataFrame(all_slides_features)
    print(f"Slides processed this run: {len(final_df)}")
    print(f"Features per slide: {final_df.shape[1]}")

    # Diagnostics
    nucleus_count_cols = [c for c in final_df.columns if 'nucleus_count' in c.lower()]
    for col in nucleus_count_cols:
        zero_count = (final_df[col] == 0).sum()
        print(f"{col}: mean={final_df[col].mean():.2f}, "
              f"zero slides={zero_count}/{len(final_df)} ({100*zero_count/len(final_df):.1f}%)")

    nan_total = final_df.isna().sum().sum()
    print(f"Total NaN values: {nan_total} ({'none ✓' if nan_total == 0 else 'check merge script'})")

    csv_path = f'{output_path}/{dataset_name}_{run_label}_features.csv'
    final_df.to_csv(csv_path, index=False)
    print(f"\n✓ Saved: {csv_path}")
else:
    print("⚠ No slides were successfully processed this run!")

# ============================================================================
# STEP 10: UPLOAD THIS RUN'S OUTPUT TO KAGGLE DATASETS
# ============================================================================

from kaggle import api

dataset_slug = f"object-features-{dataset_name.replace('_', '-')}-{run_label.replace('_', '-')}"
metadata = {
    "title": f"Object Features {dataset_name} {run_label}",
    "id": f"onyekamuoka/{dataset_slug}",
    "licenses": [{"name": "CC0-1.0"}]
}

with open(f'{output_path}/dataset-metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

try:
    api.dataset_create_new(
        folder=output_path, dir_mode='zip', convert_to_csv=False, public=False
    )
    print(f"✓ Created Kaggle dataset: {dataset_slug}")
except Exception:
    api.dataset_create_version(
        folder=output_path,
        version_notes=f"Run {run_label}",
        dir_mode='zip', convert_to_csv=False
    )
    print(f"✓ Updated Kaggle dataset: {dataset_slug}")

print(f"\n✓ https://www.kaggle.com/datasets/onyekamuoka/{dataset_slug}")
print("\n" + "="*80)
print(f"RUN COMPLETE — {run_label}")
print("Change SLIDE_START_INDEX and SLIDE_END_INDEX and run again for next slice.")
print("When all runs done, use bracs_wsi_merge.py to combine all CSVs.")
print("="*80)

# ============================================================================
# KAGGLE NOTEBOOK: BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION (SLICED RUNS)
# ============================================================================
#
# HOW TO USE:
#   This notebook processes a specific slice of slides to stay within
#   Kaggle's 12-hour time limit. Run it multiple times, changing only
#   SLIDE_START_INDEX and SLIDE_END_INDEX each time.
#
#   Recommended splits for UDH (74 slides, 31 already done):
#   ┌─────────┬─────────────────────┬──────────────────┐
#   │  Run    │  Slide Range        │  Change To       │
#   ├─────────┼─────────────────────┼──────────────────┤
#   │  Run A  │  slides 32 – 47    │  START=32 END=48 │
#   │  Run B  │  slides 48 – 63    │  START=48 END=64 │
#   │  Run C  │  slides 64 – 74    │  START=64 END=74 │
#   └─────────┴─────────────────────┴──────────────────┘
#
#   After all runs complete, use bracs_wsi_merge.py to combine the CSVs.
#
# ============================================================================

import os
import numpy as np
import cv2
import kagglehub
import openslide
from skimage.color import rgb2hsv
from skimage.morphology import remove_small_holes, remove_small_objects, disk, dilation
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from scipy.spatial.distance import cdist
from scipy.stats import skew, kurtosis
import json
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# *** CHANGE THESE TWO NUMBERS EACH RUN ***
# ============================================================================

SLIDE_START_INDEX = 15   # first slide to process (0-based index into sorted slide list)
SLIDE_END_INDEX   = 30   # process up to but NOT including this index

# ============================================================================
# CONFIGURATION (leave these unchanged across all runs)
# ============================================================================

PARALLEL_WORKERS = 4
BATCH_SIZE = 8
dataset_to_process = "saadinn/bracs-wsi-group-mt-type-ic-01"

# ============================================================================
# SETUP
# ============================================================================

print("="*80)
print("BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION - SLICED RUN")
print("="*80)
print(f"\nSlide slice: index {SLIDE_START_INDEX} → {SLIDE_END_INDEX - 1}")
print(f"Downloading dataset: {dataset_to_process}")

dataset_path = kagglehub.dataset_download(dataset_to_process)
dataset_name = dataset_to_process.split('/')[-1]

print(f"✓ Downloaded to: {dataset_path}")
print(f"✓ Dataset name:  {dataset_name}")

# ============================================================================
# STEP 2: TISSUE DETECTION
# ============================================================================

def build_tissue_mask(slide):
    mask_level = slide.level_count - 1
    img = np.array(
        slide.read_region((0, 0), mask_level, slide.level_dimensions[mask_level])
    )[:, :, :3]

    hsv = rgb2hsv(img)
    saturation = hsv[:, :, 1]
    value = hsv[:, :, 2]

    mask = (saturation > 0.03) & (value < 0.95)
    mask = remove_small_objects(mask, min_size=300)
    mask = remove_small_holes(mask, area_threshold=300)
    return mask, mask_level


def tissue_ratio(mask, x_level0, y_level0, patch_size_level0, scale):
    xs = int(x_level0 / scale)
    ys = int(y_level0 / scale)
    ps = max(int(patch_size_level0 / scale), 1)
    patch_mask = mask[ys:ys + ps, xs:xs + ps]
    if patch_mask.size == 0:
        return 0.0
    return float(patch_mask.mean())

# ============================================================================
# STEP 3: NUCLEUS SEGMENTATION
# ============================================================================

class NucleusSegmenter:

    @staticmethod
    def extract_hematoxylin_channel(rgb_patch):
        rgb_norm = rgb_patch.astype(np.float32) / 255.0
        rgb_norm = np.clip(rgb_norm, 1e-6, 1.0)
        od = -np.log(rgb_norm)

        HE_matrix = np.array([
            [0.65, 0.70, 0.29],
            [0.07, 0.99, 0.11],
            [0.27, 0.57, 0.78]
        ])

        od_flat = od.reshape(-1, 3)
        stains = np.linalg.lstsq(HE_matrix.T, od_flat.T, rcond=None)[0].T
        hematoxylin = stains[:, 0].reshape(rgb_patch.shape[:2])
        hematoxylin = np.clip(hematoxylin, 0, None)
        hematoxylin = (hematoxylin / (hematoxylin.max() + 1e-6) * 255).astype(np.uint8)
        return hematoxylin

    @staticmethod
    def segment_nuclei(rgb_patch):
        try:
            hematoxylin = NucleusSegmenter.extract_hematoxylin_channel(rgb_patch)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            hematoxylin_eq = clahe.apply(hematoxylin)

            _, nucleus_mask_otsu = cv2.threshold(
                hematoxylin_eq, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
            )

            otsu_ratio = nucleus_mask_otsu.mean() / 255.0
            if otsu_ratio < 0.05 or otsu_ratio > 0.70:
                nucleus_mask = cv2.adaptiveThreshold(
                    hematoxylin_eq, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY,
                    blockSize=31,
                    C=-2
                )
                nucleus_mask = nucleus_mask > 0
            else:
                nucleus_mask = nucleus_mask_otsu > 0

        except Exception:
            gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
            gray_inv = 255 - gray
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            gray_inv_eq = clahe.apply(gray_inv)
            nucleus_mask = cv2.adaptiveThreshold(
                gray_inv_eq, 255,
                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY,
                blockSize=31,
                C=-2
            )
            nucleus_mask = nucleus_mask > 0

        nucleus_mask = remove_small_objects(nucleus_mask, min_size=8)
        nucleus_mask = remove_small_holes(nucleus_mask, area_threshold=8)

        if not nucleus_mask.any():
            return np.zeros(rgb_patch.shape[:2], dtype=int)

        nucleus_mask = dilation(nucleus_mask, disk(1))
        distance = ndi.distance_transform_edt(nucleus_mask)
        coords = peak_local_max(
            distance, min_distance=2, labels=nucleus_mask, exclude_border=False
        )

        if len(coords) == 0:
            return label(nucleus_mask)

        mask_centers = np.zeros(distance.shape, dtype=bool)
        mask_centers[tuple(coords.T)] = True
        markers = label(mask_centers)
        labeled_nuclei = watershed(-distance, markers, mask=nucleus_mask)

        return labeled_nuclei

# ============================================================================
# STEP 4: FEATURE EXTRACTORS
# ============================================================================

class NucleusFeatureExtractor:

    @staticmethod
    def extract_nucleus_features(region, intensity_image):
        features = {}
        features['area'] = float(region.area)
        features['perimeter'] = float(region.perimeter)
        features['eccentricity'] = float(region.eccentricity)
        features['solidity'] = float(region.solidity)
        features['extent'] = float(region.extent)
        features['orientation'] = float(region.orientation)
        features['circularity'] = (
            float(4 * np.pi * region.area / (region.perimeter ** 2))
            if region.perimeter > 0 else 0.0
        )
        features['major_axis_length'] = float(region.major_axis_length)
        features['minor_axis_length'] = float(region.minor_axis_length)
        features['aspect_ratio'] = (
            float(region.major_axis_length / region.minor_axis_length)
            if region.minor_axis_length > 0 else 0.0
        )
        bbox = region.bbox
        features['bbox_width'] = float(bbox[3] - bbox[1])
        features['bbox_height'] = float(bbox[2] - bbox[0])
        features['bbox_area'] = float(features['bbox_width'] * features['bbox_height'])

        nucleus_pixels = intensity_image[region.coords[:, 0], region.coords[:, 1]]
        features['intensity_mean'] = float(np.mean(nucleus_pixels))
        features['intensity_std'] = float(np.std(nucleus_pixels))
        features['intensity_min'] = float(np.min(nucleus_pixels))
        features['intensity_max'] = float(np.max(nucleus_pixels))
        features['intensity_range'] = float(np.max(nucleus_pixels) - np.min(nucleus_pixels))
        features['intensity_median'] = float(np.median(nucleus_pixels))
        features['intensity_skewness'] = float(skew(nucleus_pixels))
        features['intensity_kurtosis'] = float(kurtosis(nucleus_pixels))
        return features


class SpatialFeatureExtractor:

    @staticmethod
    def extract_spatial_features(regions):
        default_keys = [
            'nucleus_count', 'total_nucleus_area',
            'nn_distance_mean', 'nn_distance_std', 'nn_distance_min',
            'nn_distance_max', 'nn_distance_median',
            'pairwise_distance_mean', 'pairwise_distance_std',
            'pairwise_distance_min', 'pairwise_distance_max',
            'centroid_x_variance', 'centroid_y_variance',
            'centroid_x_range', 'centroid_y_range',
        ]
        features = {k: 0.0 for k in default_keys}

        if len(regions) == 0:
            return features

        centroids = np.array([region.centroid for region in regions])
        features['nucleus_count'] = float(len(regions))
        features['total_nucleus_area'] = float(sum(r.area for r in regions))

        if len(centroids) > 1:
            distances = cdist(centroids, centroids)
            np.fill_diagonal(distances, np.inf)
            nn_distances = np.min(distances, axis=1)
            features['nn_distance_mean'] = float(np.mean(nn_distances))
            features['nn_distance_std'] = float(np.std(nn_distances))
            features['nn_distance_min'] = float(np.min(nn_distances))
            features['nn_distance_max'] = float(np.max(nn_distances))
            features['nn_distance_median'] = float(np.median(nn_distances))

            upper_triangle = distances[np.triu_indices_from(distances, k=1)]
            finite_vals = upper_triangle[np.isfinite(upper_triangle)]
            if len(finite_vals) > 0:
                features['pairwise_distance_mean'] = float(np.mean(finite_vals))
                features['pairwise_distance_std'] = float(np.std(finite_vals))
                features['pairwise_distance_min'] = float(np.min(finite_vals))
                features['pairwise_distance_max'] = float(np.max(finite_vals))

        if len(centroids) > 2:
            features['centroid_x_variance'] = float(np.var(centroids[:, 1]))
            features['centroid_y_variance'] = float(np.var(centroids[:, 0]))
            features['centroid_x_range'] = float(np.max(centroids[:, 1]) - np.min(centroids[:, 1]))
            features['centroid_y_range'] = float(np.max(centroids[:, 0]) - np.min(centroids[:, 0]))

        return features


class GraphFeatureExtractor:

    @staticmethod
    def build_delaunay_graph(centroids, max_edge_length=100):
        from scipy.spatial import Delaunay
        if len(centroids) < 3:
            return None, None
        try:
            tri = Delaunay(centroids)
            edges = set()
            for simplex in tri.simplices:
                for i in range(3):
                    edge = tuple(sorted([simplex[i], simplex[(i + 1) % 3]]))
                    dist = np.linalg.norm(centroids[edge[0]] - centroids[edge[1]])
                    if dist < max_edge_length:
                        edges.add(edge)
            return tri, list(edges)
        except Exception:
            return None, None

    @staticmethod
    def extract_graph_features(regions):
        default_keys = [
            'num_edges', 'edge_density',
            'degree_mean', 'degree_std', 'degree_max', 'degree_min',
            'edge_length_mean', 'edge_length_std',
            'edge_length_min', 'edge_length_max',
        ]
        features = {k: 0.0 for k in default_keys}

        if len(regions) < 3:
            return features

        centroids = np.array([region.centroid for region in regions])
        tri, edges = GraphFeatureExtractor.build_delaunay_graph(centroids)

        if edges is None or len(edges) == 0:
            return features

        features['num_edges'] = float(len(edges))
        n = len(regions)
        features['edge_density'] = float(len(edges) / (n * (n - 1) / 2))

        degree_count = {}
        for edge in edges:
            degree_count[edge[0]] = degree_count.get(edge[0], 0) + 1
            degree_count[edge[1]] = degree_count.get(edge[1], 0) + 1

        if degree_count:
            degrees = list(degree_count.values())
            features['degree_mean'] = float(np.mean(degrees))
            features['degree_std'] = float(np.std(degrees))
            features['degree_max'] = float(np.max(degrees))
            features['degree_min'] = float(np.min(degrees))

        edge_lengths = [
            np.linalg.norm(centroids[e[0]] - centroids[e[1]]) for e in edges
        ]
        features['edge_length_mean'] = float(np.mean(edge_lengths))
        features['edge_length_std'] = float(np.std(edge_lengths))
        features['edge_length_min'] = float(np.min(edge_lengths))
        features['edge_length_max'] = float(np.max(edge_lengths))

        return features


class ComprehensiveObjectFeatureExtractor:

    def __init__(self):
        self.nucleus_extractor = NucleusFeatureExtractor()
        self.spatial_extractor = SpatialFeatureExtractor()
        self.graph_extractor = GraphFeatureExtractor()

    def extract_all_features(self, rgb_patch):
        all_features = {}

        labeled_nuclei = NucleusSegmenter.segment_nuclei(rgb_patch)
        gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
        regions = regionprops(labeled_nuclei, intensity_image=gray)
        regions = [r for r in regions if r.area >= 8]

        all_features['nucleus_count'] = len(regions)
        all_features['total_nucleus_area'] = float(sum(r.area for r in regions)) if regions else 0.0

        if len(regions) > 0:
            nucleus_features_list = []
            for region in regions:
                try:
                    nuc_feat = self.nucleus_extractor.extract_nucleus_features(region, gray)
                    nucleus_features_list.append(nuc_feat)
                except Exception:
                    continue

            if nucleus_features_list:
                df = pd.DataFrame(nucleus_features_list)
                for col in df.columns:
                    all_features[f'nucleus_{col}_mean'] = float(df[col].mean())
                    all_features[f'nucleus_{col}_std'] = float(df[col].std())
                    all_features[f'nucleus_{col}_min'] = float(df[col].min())
                    all_features[f'nucleus_{col}_max'] = float(df[col].max())
                    all_features[f'nucleus_{col}_median'] = float(df[col].median())

        spatial_features = self.spatial_extractor.extract_spatial_features(regions)
        all_features.update(spatial_features)

        graph_features = self.graph_extractor.extract_graph_features(regions)
        all_features.update(graph_features)

        return all_features

# ============================================================================
# STEP 5: WSI PATCH EXTRACTION
# ============================================================================

def extract_object_features_from_wsi(wsi_path, patch_size=256, target_level=1, tissue_threshold=0.3):
    slide = openslide.OpenSlide(wsi_path)
    tissue_mask, mask_level = build_tissue_mask(slide)
    mask_scale = slide.level_downsamples[mask_level]
    target_downsample = slide.level_downsamples[target_level]

    extractor = ComprehensiveObjectFeatureExtractor()
    features_list = []
    width, height = slide.level_dimensions[target_level]

    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            x_level0 = int(x * target_downsample)
            y_level0 = int(y * target_downsample)
            patch_size_level0 = int(patch_size * target_downsample)

            ratio = tissue_ratio(tissue_mask, x_level0, y_level0, patch_size_level0, mask_scale)

            if ratio < tissue_threshold:
                continue

            patch = slide.read_region((x_level0, y_level0), target_level, (patch_size, patch_size))
            patch = np.array(patch)[:, :, :3]

            try:
                patch_features = extractor.extract_all_features(patch)
                patch_features['patch_x'] = x
                patch_features['patch_y'] = y
                patch_features['tissue_ratio'] = float(ratio)
                features_list.append(patch_features)
            except Exception:
                continue

    slide.close()
    return features_list

# ============================================================================
# STEP 6: SLIDE-LEVEL AGGREGATION
# ============================================================================

def aggregate_patch_features_to_slide_level(patch_features_list, slide_name):
    if not patch_features_list:
        return {}

    df = pd.DataFrame(patch_features_list)
    metadata_cols = ['patch_x', 'patch_y', 'tissue_ratio']
    feature_cols = [col for col in df.columns if col not in metadata_cols]

    slide_features = {'slide_name': slide_name, 'num_patches': len(patch_features_list)}

    for feature in feature_cols:
        feature_values = df[feature].dropna().values
        if len(feature_values) > 0:
            slide_features[f'{feature}_mean'] = float(np.mean(feature_values))
            slide_features[f'{feature}_std'] = float(np.std(feature_values))
            slide_features[f'{feature}_min'] = float(np.min(feature_values))
            slide_features[f'{feature}_max'] = float(np.max(feature_values))
            slide_features[f'{feature}_median'] = float(np.median(feature_values))
            slide_features[f'{feature}_q25'] = float(np.percentile(feature_values, 25))
            slide_features[f'{feature}_q75'] = float(np.percentile(feature_values, 75))
        else:
            for suffix in ['mean', 'std', 'min', 'max', 'median', 'q25', 'q75']:
                slide_features[f'{feature}_{suffix}'] = 0.0

    slide_features['tissue_ratio_mean'] = float(df['tissue_ratio'].mean())
    slide_features['tissue_ratio_std'] = float(df['tissue_ratio'].std())

    return slide_features

# ============================================================================
# STEP 7: PARALLEL SLIDE PROCESSING
# ============================================================================

def process_single_slide(args):
    slide_path, dataset_name, patch_size, target_level, tissue_threshold = args
    slide_name = os.path.basename(slide_path)
    try:
        patch_features = extract_object_features_from_wsi(
            slide_path, patch_size, target_level, tissue_threshold
        )
        if patch_features:
            slide_features = aggregate_patch_features_to_slide_level(
                patch_features, f"{dataset_name}_{slide_name}"
            )
            return {
                'success': True, 'slide_path': slide_path,
                'slide_name': slide_name, 'features': slide_features,
                'num_patches': len(patch_features), 'error': None
            }
        else:
            return {
                'success': False, 'slide_path': slide_path,
                'slide_name': slide_name, 'features': None,
                'num_patches': 0, 'error': 'No tissue patches found'
            }
    except Exception as e:
        return {
            'success': False, 'slide_path': slide_path,
            'slide_name': slide_name, 'features': None,
            'num_patches': 0, 'error': str(e)
        }

# ============================================================================
# STEP 8: MAIN PROCESSING LOOP
# ============================================================================

print("\n" + "="*80)
print("STARTING SLICED BATCH PROCESSING")
print("="*80)

# Build run-specific output folder so each run's CSV is separate
run_label = f"slides_{SLIDE_START_INDEX}_{SLIDE_END_INDEX - 1}"
output_path = f'/kaggle/working/object_features_{dataset_name}_{run_label}/'
os.makedirs(output_path, exist_ok=True)

# Collect and sort all slides so the index is deterministic across runs
all_slides = []
for root, dirs, files in os.walk(dataset_path):
    all_slides.extend([
        os.path.join(root, f) for f in files
        if f.endswith(('.svs', '.tif', '.tiff', '.ndpi'))
    ])
all_slides = sorted(all_slides)  # IMPORTANT: sort so index is consistent every run

print(f"Total slides in dataset: {len(all_slides)}")

# Apply the slice
slides_to_process = all_slides[SLIDE_START_INDEX:SLIDE_END_INDEX]
print(f"This run will process: {len(slides_to_process)} slides "
      f"(index {SLIDE_START_INDEX} → {SLIDE_END_INDEX - 1})")
for s in slides_to_process:
    print(f"  • {os.path.basename(s)}")

all_slides_features = []
total_processed = 0

for batch_start in range(0, len(slides_to_process), BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, len(slides_to_process))
    batch_slides = slides_to_process[batch_start:batch_end]

    print(f"\n{'='*80}")
    print(f"BATCH {batch_start // BATCH_SIZE + 1} — "
          f"slides {SLIDE_START_INDEX + batch_start + 1}–"
          f"{SLIDE_START_INDEX + batch_end} of {len(all_slides)} total")
    print(f"{'='*80}")

    slide_args = [(s, dataset_name, 256, 1, 0.3) for s in batch_slides]

    with ProcessPoolExecutor(max_workers=PARALLEL_WORKERS) as executor:
        futures = {executor.submit(process_single_slide, args): args[0] for args in slide_args}

        for future in as_completed(futures):
            slide_path = futures[future]
            try:
                result = future.result()
                if result['success']:
                    print(f"  ✓ {result['slide_name']}: {result['num_patches']} patches")
                    all_slides_features.append(result['features'])
                    total_processed += 1
                else:
                    print(f"  ✗ {result['slide_name']}: {result['error']}")
            except Exception as e:
                print(f"  ✗ Error with {os.path.basename(slide_path)}: {str(e)}")

    # Save interim CSV after every batch
    if all_slides_features:
        temp_df = pd.DataFrame(all_slides_features)
        temp_df.to_csv(f'{output_path}/slide_object_features_temp.csv', index=False)
        print(f"\n  💾 Batch checkpoint: {total_processed} slides saved so far")

# ============================================================================
# STEP 9: SAVE THIS RUN'S FINAL CSV
# ============================================================================

print("\n" + "="*80)
print("SAVING RUN RESULTS")
print("="*80)

if all_slides_features:
    final_df = pd.DataFrame(all_slides_features)
    print(f"Slides processed this run: {len(final_df)}")
    print(f"Features per slide: {final_df.shape[1]}")

    # Diagnostics
    nucleus_count_cols = [c for c in final_df.columns if 'nucleus_count' in c.lower()]
    for col in nucleus_count_cols:
        zero_count = (final_df[col] == 0).sum()
        print(f"{col}: mean={final_df[col].mean():.2f}, "
              f"zero slides={zero_count}/{len(final_df)} ({100*zero_count/len(final_df):.1f}%)")

    nan_total = final_df.isna().sum().sum()
    print(f"Total NaN values: {nan_total} ({'none ✓' if nan_total == 0 else 'check merge script'})")

    csv_path = f'{output_path}/{dataset_name}_{run_label}_features.csv'
    final_df.to_csv(csv_path, index=False)
    print(f"\n✓ Saved: {csv_path}")
else:
    print("⚠ No slides were successfully processed this run!")

# ============================================================================
# STEP 10: UPLOAD THIS RUN'S OUTPUT TO KAGGLE DATASETS
# ============================================================================

from kaggle import api

dataset_slug = f"object-features-{dataset_name.replace('_', '-')}-{run_label.replace('_', '-')}"
metadata = {
    "title": f"Object Features {dataset_name} {run_label}",
    "id": f"onyekamuoka/{dataset_slug}",
    "licenses": [{"name": "CC0-1.0"}]
}

with open(f'{output_path}/dataset-metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

try:
    api.dataset_create_new(
        folder=output_path, dir_mode='zip', convert_to_csv=False, public=False
    )
    print(f"✓ Created Kaggle dataset: {dataset_slug}")
except Exception:
    api.dataset_create_version(
        folder=output_path,
        version_notes=f"Run {run_label}",
        dir_mode='zip', convert_to_csv=False
    )
    print(f"✓ Updated Kaggle dataset: {dataset_slug}")

print(f"\n✓ https://www.kaggle.com/datasets/onyekamuoka/{dataset_slug}")
print("\n" + "="*80)
print(f"RUN COMPLETE — {run_label}")
print("Change SLIDE_START_INDEX and SLIDE_END_INDEX and run again for next slice.")
print("When all runs done, use bracs_wsi_merge.py to combine all CSVs.")
print("="*80)

# ============================================================================
# KAGGLE NOTEBOOK: BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION (SLICED RUNS)
# ============================================================================
#
# HOW TO USE:
#   This notebook processes a specific slice of slides to stay within
#   Kaggle's 12-hour time limit. Run it multiple times, changing only
#   SLIDE_START_INDEX and SLIDE_END_INDEX each time.
#
#   Recommended splits for UDH (74 slides, 31 already done):
#   ┌─────────┬─────────────────────┬──────────────────┐
#   │  Run    │  Slide Range        │  Change To       │
#   ├─────────┼─────────────────────┼──────────────────┤
#   │  Run A  │  slides 32 – 47    │  START=32 END=48 │
#   │  Run B  │  slides 48 – 63    │  START=48 END=64 │
#   │  Run C  │  slides 64 – 74    │  START=64 END=74 │
#   └─────────┴─────────────────────┴──────────────────┘
#
#   After all runs complete, use bracs_wsi_merge.py to combine the CSVs.
#
# ============================================================================

import os
import numpy as np
import cv2
import kagglehub
import openslide
from skimage.color import rgb2hsv
from skimage.morphology import remove_small_holes, remove_small_objects, disk, dilation
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from scipy.spatial.distance import cdist
from scipy.stats import skew, kurtosis
import json
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# *** CHANGE THESE TWO NUMBERS EACH RUN ***
# ============================================================================

SLIDE_START_INDEX = 15   # first slide to process (0-based index into sorted slide list)
SLIDE_END_INDEX   = 22   # process up to but NOT including this index

# ============================================================================
# CONFIGURATION (leave these unchanged across all runs)
# ============================================================================

PARALLEL_WORKERS = 4
BATCH_SIZE = 8
dataset_to_process = "saadinn/bracs-wsi-group-mt-type-ic-01"

# ============================================================================
# SETUP
# ============================================================================

print("="*80)
print("BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION - SLICED RUN")
print("="*80)
print(f"\nSlide slice: index {SLIDE_START_INDEX} → {SLIDE_END_INDEX - 1}")
print(f"Downloading dataset: {dataset_to_process}")

dataset_path = kagglehub.dataset_download(dataset_to_process)
dataset_name = dataset_to_process.split('/')[-1]

print(f"✓ Downloaded to: {dataset_path}")
print(f"✓ Dataset name:  {dataset_name}")

# ============================================================================
# STEP 2: TISSUE DETECTION
# ============================================================================

def build_tissue_mask(slide):
    mask_level = slide.level_count - 1
    img = np.array(
        slide.read_region((0, 0), mask_level, slide.level_dimensions[mask_level])
    )[:, :, :3]

    hsv = rgb2hsv(img)
    saturation = hsv[:, :, 1]
    value = hsv[:, :, 2]

    mask = (saturation > 0.03) & (value < 0.95)
    mask = remove_small_objects(mask, min_size=300)
    mask = remove_small_holes(mask, area_threshold=300)
    return mask, mask_level


def tissue_ratio(mask, x_level0, y_level0, patch_size_level0, scale):
    xs = int(x_level0 / scale)
    ys = int(y_level0 / scale)
    ps = max(int(patch_size_level0 / scale), 1)
    patch_mask = mask[ys:ys + ps, xs:xs + ps]
    if patch_mask.size == 0:
        return 0.0
    return float(patch_mask.mean())

# ============================================================================
# STEP 3: NUCLEUS SEGMENTATION
# ============================================================================

class NucleusSegmenter:

    @staticmethod
    def extract_hematoxylin_channel(rgb_patch):
        rgb_norm = rgb_patch.astype(np.float32) / 255.0
        rgb_norm = np.clip(rgb_norm, 1e-6, 1.0)
        od = -np.log(rgb_norm)

        HE_matrix = np.array([
            [0.65, 0.70, 0.29],
            [0.07, 0.99, 0.11],
            [0.27, 0.57, 0.78]
        ])

        od_flat = od.reshape(-1, 3)
        stains = np.linalg.lstsq(HE_matrix.T, od_flat.T, rcond=None)[0].T
        hematoxylin = stains[:, 0].reshape(rgb_patch.shape[:2])
        hematoxylin = np.clip(hematoxylin, 0, None)
        hematoxylin = (hematoxylin / (hematoxylin.max() + 1e-6) * 255).astype(np.uint8)
        return hematoxylin

    @staticmethod
    def segment_nuclei(rgb_patch):
        try:
            hematoxylin = NucleusSegmenter.extract_hematoxylin_channel(rgb_patch)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            hematoxylin_eq = clahe.apply(hematoxylin)

            _, nucleus_mask_otsu = cv2.threshold(
                hematoxylin_eq, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
            )

            otsu_ratio = nucleus_mask_otsu.mean() / 255.0
            if otsu_ratio < 0.05 or otsu_ratio > 0.70:
                nucleus_mask = cv2.adaptiveThreshold(
                    hematoxylin_eq, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY,
                    blockSize=31,
                    C=-2
                )
                nucleus_mask = nucleus_mask > 0
            else:
                nucleus_mask = nucleus_mask_otsu > 0

        except Exception:
            gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
            gray_inv = 255 - gray
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            gray_inv_eq = clahe.apply(gray_inv)
            nucleus_mask = cv2.adaptiveThreshold(
                gray_inv_eq, 255,
                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY,
                blockSize=31,
                C=-2
            )
            nucleus_mask = nucleus_mask > 0

        nucleus_mask = remove_small_objects(nucleus_mask, min_size=8)
        nucleus_mask = remove_small_holes(nucleus_mask, area_threshold=8)

        if not nucleus_mask.any():
            return np.zeros(rgb_patch.shape[:2], dtype=int)

        nucleus_mask = dilation(nucleus_mask, disk(1))
        distance = ndi.distance_transform_edt(nucleus_mask)
        coords = peak_local_max(
            distance, min_distance=2, labels=nucleus_mask, exclude_border=False
        )

        if len(coords) == 0:
            return label(nucleus_mask)

        mask_centers = np.zeros(distance.shape, dtype=bool)
        mask_centers[tuple(coords.T)] = True
        markers = label(mask_centers)
        labeled_nuclei = watershed(-distance, markers, mask=nucleus_mask)

        return labeled_nuclei

# ============================================================================
# STEP 4: FEATURE EXTRACTORS
# ============================================================================

class NucleusFeatureExtractor:

    @staticmethod
    def extract_nucleus_features(region, intensity_image):
        features = {}
        features['area'] = float(region.area)
        features['perimeter'] = float(region.perimeter)
        features['eccentricity'] = float(region.eccentricity)
        features['solidity'] = float(region.solidity)
        features['extent'] = float(region.extent)
        features['orientation'] = float(region.orientation)
        features['circularity'] = (
            float(4 * np.pi * region.area / (region.perimeter ** 2))
            if region.perimeter > 0 else 0.0
        )
        features['major_axis_length'] = float(region.major_axis_length)
        features['minor_axis_length'] = float(region.minor_axis_length)
        features['aspect_ratio'] = (
            float(region.major_axis_length / region.minor_axis_length)
            if region.minor_axis_length > 0 else 0.0
        )
        bbox = region.bbox
        features['bbox_width'] = float(bbox[3] - bbox[1])
        features['bbox_height'] = float(bbox[2] - bbox[0])
        features['bbox_area'] = float(features['bbox_width'] * features['bbox_height'])

        nucleus_pixels = intensity_image[region.coords[:, 0], region.coords[:, 1]]
        features['intensity_mean'] = float(np.mean(nucleus_pixels))
        features['intensity_std'] = float(np.std(nucleus_pixels))
        features['intensity_min'] = float(np.min(nucleus_pixels))
        features['intensity_max'] = float(np.max(nucleus_pixels))
        features['intensity_range'] = float(np.max(nucleus_pixels) - np.min(nucleus_pixels))
        features['intensity_median'] = float(np.median(nucleus_pixels))
        features['intensity_skewness'] = float(skew(nucleus_pixels))
        features['intensity_kurtosis'] = float(kurtosis(nucleus_pixels))
        return features


class SpatialFeatureExtractor:

    @staticmethod
    def extract_spatial_features(regions):
        default_keys = [
            'nucleus_count', 'total_nucleus_area',
            'nn_distance_mean', 'nn_distance_std', 'nn_distance_min',
            'nn_distance_max', 'nn_distance_median',
            'pairwise_distance_mean', 'pairwise_distance_std',
            'pairwise_distance_min', 'pairwise_distance_max',
            'centroid_x_variance', 'centroid_y_variance',
            'centroid_x_range', 'centroid_y_range',
        ]
        features = {k: 0.0 for k in default_keys}

        if len(regions) == 0:
            return features

        centroids = np.array([region.centroid for region in regions])
        features['nucleus_count'] = float(len(regions))
        features['total_nucleus_area'] = float(sum(r.area for r in regions))

        if len(centroids) > 1:
            distances = cdist(centroids, centroids)
            np.fill_diagonal(distances, np.inf)
            nn_distances = np.min(distances, axis=1)
            features['nn_distance_mean'] = float(np.mean(nn_distances))
            features['nn_distance_std'] = float(np.std(nn_distances))
            features['nn_distance_min'] = float(np.min(nn_distances))
            features['nn_distance_max'] = float(np.max(nn_distances))
            features['nn_distance_median'] = float(np.median(nn_distances))

            upper_triangle = distances[np.triu_indices_from(distances, k=1)]
            finite_vals = upper_triangle[np.isfinite(upper_triangle)]
            if len(finite_vals) > 0:
                features['pairwise_distance_mean'] = float(np.mean(finite_vals))
                features['pairwise_distance_std'] = float(np.std(finite_vals))
                features['pairwise_distance_min'] = float(np.min(finite_vals))
                features['pairwise_distance_max'] = float(np.max(finite_vals))

        if len(centroids) > 2:
            features['centroid_x_variance'] = float(np.var(centroids[:, 1]))
            features['centroid_y_variance'] = float(np.var(centroids[:, 0]))
            features['centroid_x_range'] = float(np.max(centroids[:, 1]) - np.min(centroids[:, 1]))
            features['centroid_y_range'] = float(np.max(centroids[:, 0]) - np.min(centroids[:, 0]))

        return features


class GraphFeatureExtractor:

    @staticmethod
    def build_delaunay_graph(centroids, max_edge_length=100):
        from scipy.spatial import Delaunay
        if len(centroids) < 3:
            return None, None
        try:
            tri = Delaunay(centroids)
            edges = set()
            for simplex in tri.simplices:
                for i in range(3):
                    edge = tuple(sorted([simplex[i], simplex[(i + 1) % 3]]))
                    dist = np.linalg.norm(centroids[edge[0]] - centroids[edge[1]])
                    if dist < max_edge_length:
                        edges.add(edge)
            return tri, list(edges)
        except Exception:
            return None, None

    @staticmethod
    def extract_graph_features(regions):
        default_keys = [
            'num_edges', 'edge_density',
            'degree_mean', 'degree_std', 'degree_max', 'degree_min',
            'edge_length_mean', 'edge_length_std',
            'edge_length_min', 'edge_length_max',
        ]
        features = {k: 0.0 for k in default_keys}

        if len(regions) < 3:
            return features

        centroids = np.array([region.centroid for region in regions])
        tri, edges = GraphFeatureExtractor.build_delaunay_graph(centroids)

        if edges is None or len(edges) == 0:
            return features

        features['num_edges'] = float(len(edges))
        n = len(regions)
        features['edge_density'] = float(len(edges) / (n * (n - 1) / 2))

        degree_count = {}
        for edge in edges:
            degree_count[edge[0]] = degree_count.get(edge[0], 0) + 1
            degree_count[edge[1]] = degree_count.get(edge[1], 0) + 1

        if degree_count:
            degrees = list(degree_count.values())
            features['degree_mean'] = float(np.mean(degrees))
            features['degree_std'] = float(np.std(degrees))
            features['degree_max'] = float(np.max(degrees))
            features['degree_min'] = float(np.min(degrees))

        edge_lengths = [
            np.linalg.norm(centroids[e[0]] - centroids[e[1]]) for e in edges
        ]
        features['edge_length_mean'] = float(np.mean(edge_lengths))
        features['edge_length_std'] = float(np.std(edge_lengths))
        features['edge_length_min'] = float(np.min(edge_lengths))
        features['edge_length_max'] = float(np.max(edge_lengths))

        return features


class ComprehensiveObjectFeatureExtractor:

    def __init__(self):
        self.nucleus_extractor = NucleusFeatureExtractor()
        self.spatial_extractor = SpatialFeatureExtractor()
        self.graph_extractor = GraphFeatureExtractor()

    def extract_all_features(self, rgb_patch):
        all_features = {}

        labeled_nuclei = NucleusSegmenter.segment_nuclei(rgb_patch)
        gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
        regions = regionprops(labeled_nuclei, intensity_image=gray)
        regions = [r for r in regions if r.area >= 8]

        all_features['nucleus_count'] = len(regions)
        all_features['total_nucleus_area'] = float(sum(r.area for r in regions)) if regions else 0.0

        if len(regions) > 0:
            nucleus_features_list = []
            for region in regions:
                try:
                    nuc_feat = self.nucleus_extractor.extract_nucleus_features(region, gray)
                    nucleus_features_list.append(nuc_feat)
                except Exception:
                    continue

            if nucleus_features_list:
                df = pd.DataFrame(nucleus_features_list)
                for col in df.columns:
                    all_features[f'nucleus_{col}_mean'] = float(df[col].mean())
                    all_features[f'nucleus_{col}_std'] = float(df[col].std())
                    all_features[f'nucleus_{col}_min'] = float(df[col].min())
                    all_features[f'nucleus_{col}_max'] = float(df[col].max())
                    all_features[f'nucleus_{col}_median'] = float(df[col].median())

        spatial_features = self.spatial_extractor.extract_spatial_features(regions)
        all_features.update(spatial_features)

        graph_features = self.graph_extractor.extract_graph_features(regions)
        all_features.update(graph_features)

        return all_features

# ============================================================================
# STEP 5: WSI PATCH EXTRACTION
# ============================================================================

def extract_object_features_from_wsi(wsi_path, patch_size=256, target_level=1, tissue_threshold=0.3):
    slide = openslide.OpenSlide(wsi_path)
    tissue_mask, mask_level = build_tissue_mask(slide)
    mask_scale = slide.level_downsamples[mask_level]
    target_downsample = slide.level_downsamples[target_level]

    extractor = ComprehensiveObjectFeatureExtractor()
    features_list = []
    width, height = slide.level_dimensions[target_level]

    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            x_level0 = int(x * target_downsample)
            y_level0 = int(y * target_downsample)
            patch_size_level0 = int(patch_size * target_downsample)

            ratio = tissue_ratio(tissue_mask, x_level0, y_level0, patch_size_level0, mask_scale)

            if ratio < tissue_threshold:
                continue

            patch = slide.read_region((x_level0, y_level0), target_level, (patch_size, patch_size))
            patch = np.array(patch)[:, :, :3]

            try:
                patch_features = extractor.extract_all_features(patch)
                patch_features['patch_x'] = x
                patch_features['patch_y'] = y
                patch_features['tissue_ratio'] = float(ratio)
                features_list.append(patch_features)
            except Exception:
                continue

    slide.close()
    return features_list

# ============================================================================
# STEP 6: SLIDE-LEVEL AGGREGATION
# ============================================================================

def aggregate_patch_features_to_slide_level(patch_features_list, slide_name):
    if not patch_features_list:
        return {}

    df = pd.DataFrame(patch_features_list)
    metadata_cols = ['patch_x', 'patch_y', 'tissue_ratio']
    feature_cols = [col for col in df.columns if col not in metadata_cols]

    slide_features = {'slide_name': slide_name, 'num_patches': len(patch_features_list)}

    for feature in feature_cols:
        feature_values = df[feature].dropna().values
        if len(feature_values) > 0:
            slide_features[f'{feature}_mean'] = float(np.mean(feature_values))
            slide_features[f'{feature}_std'] = float(np.std(feature_values))
            slide_features[f'{feature}_min'] = float(np.min(feature_values))
            slide_features[f'{feature}_max'] = float(np.max(feature_values))
            slide_features[f'{feature}_median'] = float(np.median(feature_values))
            slide_features[f'{feature}_q25'] = float(np.percentile(feature_values, 25))
            slide_features[f'{feature}_q75'] = float(np.percentile(feature_values, 75))
        else:
            for suffix in ['mean', 'std', 'min', 'max', 'median', 'q25', 'q75']:
                slide_features[f'{feature}_{suffix}'] = 0.0

    slide_features['tissue_ratio_mean'] = float(df['tissue_ratio'].mean())
    slide_features['tissue_ratio_std'] = float(df['tissue_ratio'].std())

    return slide_features

# ============================================================================
# STEP 7: PARALLEL SLIDE PROCESSING
# ============================================================================

def process_single_slide(args):
    slide_path, dataset_name, patch_size, target_level, tissue_threshold = args
    slide_name = os.path.basename(slide_path)
    try:
        patch_features = extract_object_features_from_wsi(
            slide_path, patch_size, target_level, tissue_threshold
        )
        if patch_features:
            slide_features = aggregate_patch_features_to_slide_level(
                patch_features, f"{dataset_name}_{slide_name}"
            )
            return {
                'success': True, 'slide_path': slide_path,
                'slide_name': slide_name, 'features': slide_features,
                'num_patches': len(patch_features), 'error': None
            }
        else:
            return {
                'success': False, 'slide_path': slide_path,
                'slide_name': slide_name, 'features': None,
                'num_patches': 0, 'error': 'No tissue patches found'
            }
    except Exception as e:
        return {
            'success': False, 'slide_path': slide_path,
            'slide_name': slide_name, 'features': None,
            'num_patches': 0, 'error': str(e)
        }

# ============================================================================
# STEP 8: MAIN PROCESSING LOOP
# ============================================================================

print("\n" + "="*80)
print("STARTING SLICED BATCH PROCESSING")
print("="*80)

# Build run-specific output folder so each run's CSV is separate
run_label = f"slides_{SLIDE_START_INDEX}_{SLIDE_END_INDEX - 1}"
output_path = f'/kaggle/working/object_features_{dataset_name}_{run_label}/'
os.makedirs(output_path, exist_ok=True)

# Collect and sort all slides so the index is deterministic across runs
all_slides = []
for root, dirs, files in os.walk(dataset_path):
    all_slides.extend([
        os.path.join(root, f) for f in files
        if f.endswith(('.svs', '.tif', '.tiff', '.ndpi'))
    ])
all_slides = sorted(all_slides)  # IMPORTANT: sort so index is consistent every run

print(f"Total slides in dataset: {len(all_slides)}")

# Apply the slice
slides_to_process = all_slides[SLIDE_START_INDEX:SLIDE_END_INDEX]
print(f"This run will process: {len(slides_to_process)} slides "
      f"(index {SLIDE_START_INDEX} → {SLIDE_END_INDEX - 1})")
for s in slides_to_process:
    print(f"  • {os.path.basename(s)}")

all_slides_features = []
total_processed = 0

for batch_start in range(0, len(slides_to_process), BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, len(slides_to_process))
    batch_slides = slides_to_process[batch_start:batch_end]

    print(f"\n{'='*80}")
    print(f"BATCH {batch_start // BATCH_SIZE + 1} — "
          f"slides {SLIDE_START_INDEX + batch_start + 1}–"
          f"{SLIDE_START_INDEX + batch_end} of {len(all_slides)} total")
    print(f"{'='*80}")

    slide_args = [(s, dataset_name, 256, 1, 0.3) for s in batch_slides]

    with ProcessPoolExecutor(max_workers=PARALLEL_WORKERS) as executor:
        futures = {executor.submit(process_single_slide, args): args[0] for args in slide_args}

        for future in as_completed(futures):
            slide_path = futures[future]
            try:
                result = future.result()
                if result['success']:
                    print(f"  ✓ {result['slide_name']}: {result['num_patches']} patches")
                    all_slides_features.append(result['features'])
                    total_processed += 1
                else:
                    print(f"  ✗ {result['slide_name']}: {result['error']}")
            except Exception as e:
                print(f"  ✗ Error with {os.path.basename(slide_path)}: {str(e)}")

    # Save interim CSV after every batch
    if all_slides_features:
        temp_df = pd.DataFrame(all_slides_features)
        temp_df.to_csv(f'{output_path}/slide_object_features_temp.csv', index=False)
        print(f"\n  💾 Batch checkpoint: {total_processed} slides saved so far")

# ============================================================================
# STEP 9: SAVE THIS RUN'S FINAL CSV
# ============================================================================

print("\n" + "="*80)
print("SAVING RUN RESULTS")
print("="*80)

if all_slides_features:
    final_df = pd.DataFrame(all_slides_features)
    print(f"Slides processed this run: {len(final_df)}")
    print(f"Features per slide: {final_df.shape[1]}")

    # Diagnostics
    nucleus_count_cols = [c for c in final_df.columns if 'nucleus_count' in c.lower()]
    for col in nucleus_count_cols:
        zero_count = (final_df[col] == 0).sum()
        print(f"{col}: mean={final_df[col].mean():.2f}, "
              f"zero slides={zero_count}/{len(final_df)} ({100*zero_count/len(final_df):.1f}%)")

    nan_total = final_df.isna().sum().sum()
    print(f"Total NaN values: {nan_total} ({'none ✓' if nan_total == 0 else 'check merge script'})")

    csv_path = f'{output_path}/{dataset_name}_{run_label}_features.csv'
    final_df.to_csv(csv_path, index=False)
    print(f"\n✓ Saved: {csv_path}")
else:
    print("⚠ No slides were successfully processed this run!")

# ============================================================================
# STEP 10: UPLOAD THIS RUN'S OUTPUT TO KAGGLE DATASETS
# ============================================================================

from kaggle import api

dataset_slug = f"object-features-{dataset_name.replace('_', '-')}-{run_label.replace('_', '-')}"
metadata = {
    "title": f"Object Features {dataset_name} {run_label}",
    "id": f"onyekamuoka/{dataset_slug}",
    "licenses": [{"name": "CC0-1.0"}]
}

with open(f'{output_path}/dataset-metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

try:
    api.dataset_create_new(
        folder=output_path, dir_mode='zip', convert_to_csv=False, public=False
    )
    print(f"✓ Created Kaggle dataset: {dataset_slug}")
except Exception:
    api.dataset_create_version(
        folder=output_path,
        version_notes=f"Run {run_label}",
        dir_mode='zip', convert_to_csv=False
    )
    print(f"✓ Updated Kaggle dataset: {dataset_slug}")

print(f"\n✓ https://www.kaggle.com/datasets/onyekamuoka/{dataset_slug}")
print("\n" + "="*80)
print(f"RUN COMPLETE — {run_label}")
print("Change SLIDE_START_INDEX and SLIDE_END_INDEX and run again for next slice.")
print("When all runs done, use bracs_wsi_merge.py to combine all CSVs.")
print("="*80)

In [ ]:
# ============================================================================
# KAGGLE NOTEBOOK: BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION (SLICED RUNS)
# ============================================================================
#
# HOW TO USE:
#   This notebook processes a specific slice of slides to stay within
#   Kaggle's 12-hour time limit. Run it multiple times, changing only
#   SLIDE_START_INDEX and SLIDE_END_INDEX each time.
#
#   Recommended splits for UDH (74 slides, 31 already done):
#   ┌─────────┬─────────────────────┬──────────────────┐
#   │  Run    │  Slide Range        │  Change To       │
#   ├─────────┼─────────────────────┼──────────────────┤
#   │  Run A  │  slides 32 – 47    │  START=32 END=48 │
#   │  Run B  │  slides 48 – 63    │  START=48 END=64 │
#   │  Run C  │  slides 64 – 74    │  START=64 END=74 │
#   └─────────┴─────────────────────┴──────────────────┘
#
#   After all runs complete, use bracs_wsi_merge.py to combine the CSVs.
#
# ============================================================================

import os
import numpy as np
import cv2
import kagglehub
import openslide
from skimage.color import rgb2hsv
from skimage.morphology import remove_small_holes, remove_small_objects, disk, dilation
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from scipy.spatial.distance import cdist
from scipy.stats import skew, kurtosis
import json
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# *** CHANGE THESE TWO NUMBERS EACH RUN ***
# ============================================================================

SLIDE_START_INDEX = 22   # first slide to process (0-based index into sorted slide list)
SLIDE_END_INDEX   = 32   # process up to but NOT including this index

# ============================================================================
# CONFIGURATION (leave these unchanged across all runs)
# ============================================================================

PARALLEL_WORKERS = 4
BATCH_SIZE = 8
dataset_to_process = "saadinn/bracs-wsi-group-mt-type-ic-01"

# ============================================================================
# SETUP
# ============================================================================

print("="*80)
print("BRACS WSI OBJECT-LEVEL FEATURE EXTRACTION - SLICED RUN")
print("="*80)
print(f"\nSlide slice: index {SLIDE_START_INDEX} → {SLIDE_END_INDEX - 1}")
print(f"Downloading dataset: {dataset_to_process}")

dataset_path = kagglehub.dataset_download(dataset_to_process)
dataset_name = dataset_to_process.split('/')[-1]

print(f"✓ Downloaded to: {dataset_path}")
print(f"✓ Dataset name:  {dataset_name}")

# ============================================================================
# STEP 2: TISSUE DETECTION
# ============================================================================

def build_tissue_mask(slide):
    mask_level = slide.level_count - 1
    img = np.array(
        slide.read_region((0, 0), mask_level, slide.level_dimensions[mask_level])
    )[:, :, :3]

    hsv = rgb2hsv(img)
    saturation = hsv[:, :, 1]
    value = hsv[:, :, 2]

    mask = (saturation > 0.03) & (value < 0.95)
    mask = remove_small_objects(mask, min_size=300)
    mask = remove_small_holes(mask, area_threshold=300)
    return mask, mask_level


def tissue_ratio(mask, x_level0, y_level0, patch_size_level0, scale):
    xs = int(x_level0 / scale)
    ys = int(y_level0 / scale)
    ps = max(int(patch_size_level0 / scale), 1)
    patch_mask = mask[ys:ys + ps, xs:xs + ps]
    if patch_mask.size == 0:
        return 0.0
    return float(patch_mask.mean())

# ============================================================================
# STEP 3: NUCLEUS SEGMENTATION
# ============================================================================

class NucleusSegmenter:

    @staticmethod
    def extract_hematoxylin_channel(rgb_patch):
        rgb_norm = rgb_patch.astype(np.float32) / 255.0
        rgb_norm = np.clip(rgb_norm, 1e-6, 1.0)
        od = -np.log(rgb_norm)

        HE_matrix = np.array([
            [0.65, 0.70, 0.29],
            [0.07, 0.99, 0.11],
            [0.27, 0.57, 0.78]
        ])

        od_flat = od.reshape(-1, 3)
        stains = np.linalg.lstsq(HE_matrix.T, od_flat.T, rcond=None)[0].T
        hematoxylin = stains[:, 0].reshape(rgb_patch.shape[:2])
        hematoxylin = np.clip(hematoxylin, 0, None)
        hematoxylin = (hematoxylin / (hematoxylin.max() + 1e-6) * 255).astype(np.uint8)
        return hematoxylin

    @staticmethod
    def segment_nuclei(rgb_patch):
        try:
            hematoxylin = NucleusSegmenter.extract_hematoxylin_channel(rgb_patch)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            hematoxylin_eq = clahe.apply(hematoxylin)

            _, nucleus_mask_otsu = cv2.threshold(
                hematoxylin_eq, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
            )

            otsu_ratio = nucleus_mask_otsu.mean() / 255.0
            if otsu_ratio < 0.05 or otsu_ratio > 0.70:
                nucleus_mask = cv2.adaptiveThreshold(
                    hematoxylin_eq, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY,
                    blockSize=31,
                    C=-2
                )
                nucleus_mask = nucleus_mask > 0
            else:
                nucleus_mask = nucleus_mask_otsu > 0

        except Exception:
            gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
            gray_inv = 255 - gray
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            gray_inv_eq = clahe.apply(gray_inv)
            nucleus_mask = cv2.adaptiveThreshold(
                gray_inv_eq, 255,
                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY,
                blockSize=31,
                C=-2
            )
            nucleus_mask = nucleus_mask > 0

        nucleus_mask = remove_small_objects(nucleus_mask, min_size=8)
        nucleus_mask = remove_small_holes(nucleus_mask, area_threshold=8)

        if not nucleus_mask.any():
            return np.zeros(rgb_patch.shape[:2], dtype=int)

        nucleus_mask = dilation(nucleus_mask, disk(1))
        distance = ndi.distance_transform_edt(nucleus_mask)
        coords = peak_local_max(
            distance, min_distance=2, labels=nucleus_mask, exclude_border=False
        )

        if len(coords) == 0:
            return label(nucleus_mask)

        mask_centers = np.zeros(distance.shape, dtype=bool)
        mask_centers[tuple(coords.T)] = True
        markers = label(mask_centers)
        labeled_nuclei = watershed(-distance, markers, mask=nucleus_mask)

        return labeled_nuclei

# ============================================================================
# STEP 4: FEATURE EXTRACTORS
# ============================================================================

class NucleusFeatureExtractor:

    @staticmethod
    def extract_nucleus_features(region, intensity_image):
        features = {}
        features['area'] = float(region.area)
        features['perimeter'] = float(region.perimeter)
        features['eccentricity'] = float(region.eccentricity)
        features['solidity'] = float(region.solidity)
        features['extent'] = float(region.extent)
        features['orientation'] = float(region.orientation)
        features['circularity'] = (
            float(4 * np.pi * region.area / (region.perimeter ** 2))
            if region.perimeter > 0 else 0.0
        )
        features['major_axis_length'] = float(region.major_axis_length)
        features['minor_axis_length'] = float(region.minor_axis_length)
        features['aspect_ratio'] = (
            float(region.major_axis_length / region.minor_axis_length)
            if region.minor_axis_length > 0 else 0.0
        )
        bbox = region.bbox
        features['bbox_width'] = float(bbox[3] - bbox[1])
        features['bbox_height'] = float(bbox[2] - bbox[0])
        features['bbox_area'] = float(features['bbox_width'] * features['bbox_height'])

        nucleus_pixels = intensity_image[region.coords[:, 0], region.coords[:, 1]]
        features['intensity_mean'] = float(np.mean(nucleus_pixels))
        features['intensity_std'] = float(np.std(nucleus_pixels))
        features['intensity_min'] = float(np.min(nucleus_pixels))
        features['intensity_max'] = float(np.max(nucleus_pixels))
        features['intensity_range'] = float(np.max(nucleus_pixels) - np.min(nucleus_pixels))
        features['intensity_median'] = float(np.median(nucleus_pixels))
        features['intensity_skewness'] = float(skew(nucleus_pixels))
        features['intensity_kurtosis'] = float(kurtosis(nucleus_pixels))
        return features


class SpatialFeatureExtractor:

    @staticmethod
    def extract_spatial_features(regions):
        default_keys = [
            'nucleus_count', 'total_nucleus_area',
            'nn_distance_mean', 'nn_distance_std', 'nn_distance_min',
            'nn_distance_max', 'nn_distance_median',
            'pairwise_distance_mean', 'pairwise_distance_std',
            'pairwise_distance_min', 'pairwise_distance_max',
            'centroid_x_variance', 'centroid_y_variance',
            'centroid_x_range', 'centroid_y_range',
        ]
        features = {k: 0.0 for k in default_keys}

        if len(regions) == 0:
            return features

        centroids = np.array([region.centroid for region in regions])
        features['nucleus_count'] = float(len(regions))
        features['total_nucleus_area'] = float(sum(r.area for r in regions))

        if len(centroids) > 1:
            distances = cdist(centroids, centroids)
            np.fill_diagonal(distances, np.inf)
            nn_distances = np.min(distances, axis=1)
            features['nn_distance_mean'] = float(np.mean(nn_distances))
            features['nn_distance_std'] = float(np.std(nn_distances))
            features['nn_distance_min'] = float(np.min(nn_distances))
            features['nn_distance_max'] = float(np.max(nn_distances))
            features['nn_distance_median'] = float(np.median(nn_distances))

            upper_triangle = distances[np.triu_indices_from(distances, k=1)]
            finite_vals = upper_triangle[np.isfinite(upper_triangle)]
            if len(finite_vals) > 0:
                features['pairwise_distance_mean'] = float(np.mean(finite_vals))
                features['pairwise_distance_std'] = float(np.std(finite_vals))
                features['pairwise_distance_min'] = float(np.min(finite_vals))
                features['pairwise_distance_max'] = float(np.max(finite_vals))

        if len(centroids) > 2:
            features['centroid_x_variance'] = float(np.var(centroids[:, 1]))
            features['centroid_y_variance'] = float(np.var(centroids[:, 0]))
            features['centroid_x_range'] = float(np.max(centroids[:, 1]) - np.min(centroids[:, 1]))
            features['centroid_y_range'] = float(np.max(centroids[:, 0]) - np.min(centroids[:, 0]))

        return features


class GraphFeatureExtractor:

    @staticmethod
    def build_delaunay_graph(centroids, max_edge_length=100):
        from scipy.spatial import Delaunay
        if len(centroids) < 3:
            return None, None
        try:
            tri = Delaunay(centroids)
            edges = set()
            for simplex in tri.simplices:
                for i in range(3):
                    edge = tuple(sorted([simplex[i], simplex[(i + 1) % 3]]))
                    dist = np.linalg.norm(centroids[edge[0]] - centroids[edge[1]])
                    if dist < max_edge_length:
                        edges.add(edge)
            return tri, list(edges)
        except Exception:
            return None, None

    @staticmethod
    def extract_graph_features(regions):
        default_keys = [
            'num_edges', 'edge_density',
            'degree_mean', 'degree_std', 'degree_max', 'degree_min',
            'edge_length_mean', 'edge_length_std',
            'edge_length_min', 'edge_length_max',
        ]
        features = {k: 0.0 for k in default_keys}

        if len(regions) < 3:
            return features

        centroids = np.array([region.centroid for region in regions])
        tri, edges = GraphFeatureExtractor.build_delaunay_graph(centroids)

        if edges is None or len(edges) == 0:
            return features

        features['num_edges'] = float(len(edges))
        n = len(regions)
        features['edge_density'] = float(len(edges) / (n * (n - 1) / 2))

        degree_count = {}
        for edge in edges:
            degree_count[edge[0]] = degree_count.get(edge[0], 0) + 1
            degree_count[edge[1]] = degree_count.get(edge[1], 0) + 1

        if degree_count:
            degrees = list(degree_count.values())
            features['degree_mean'] = float(np.mean(degrees))
            features['degree_std'] = float(np.std(degrees))
            features['degree_max'] = float(np.max(degrees))
            features['degree_min'] = float(np.min(degrees))

        edge_lengths = [
            np.linalg.norm(centroids[e[0]] - centroids[e[1]]) for e in edges
        ]
        features['edge_length_mean'] = float(np.mean(edge_lengths))
        features['edge_length_std'] = float(np.std(edge_lengths))
        features['edge_length_min'] = float(np.min(edge_lengths))
        features['edge_length_max'] = float(np.max(edge_lengths))

        return features


class ComprehensiveObjectFeatureExtractor:

    def __init__(self):
        self.nucleus_extractor = NucleusFeatureExtractor()
        self.spatial_extractor = SpatialFeatureExtractor()
        self.graph_extractor = GraphFeatureExtractor()

    def extract_all_features(self, rgb_patch):
        all_features = {}

        labeled_nuclei = NucleusSegmenter.segment_nuclei(rgb_patch)
        gray = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2GRAY)
        regions = regionprops(labeled_nuclei, intensity_image=gray)
        regions = [r for r in regions if r.area >= 8]

        all_features['nucleus_count'] = len(regions)
        all_features['total_nucleus_area'] = float(sum(r.area for r in regions)) if regions else 0.0

        if len(regions) > 0:
            nucleus_features_list = []
            for region in regions:
                try:
                    nuc_feat = self.nucleus_extractor.extract_nucleus_features(region, gray)
                    nucleus_features_list.append(nuc_feat)
                except Exception:
                    continue

            if nucleus_features_list:
                df = pd.DataFrame(nucleus_features_list)
                for col in df.columns:
                    all_features[f'nucleus_{col}_mean'] = float(df[col].mean())
                    all_features[f'nucleus_{col}_std'] = float(df[col].std())
                    all_features[f'nucleus_{col}_min'] = float(df[col].min())
                    all_features[f'nucleus_{col}_max'] = float(df[col].max())
                    all_features[f'nucleus_{col}_median'] = float(df[col].median())

        spatial_features = self.spatial_extractor.extract_spatial_features(regions)
        all_features.update(spatial_features)

        graph_features = self.graph_extractor.extract_graph_features(regions)
        all_features.update(graph_features)

        return all_features

# ============================================================================
# STEP 5: WSI PATCH EXTRACTION
# ============================================================================

def extract_object_features_from_wsi(wsi_path, patch_size=256, target_level=1, tissue_threshold=0.3):
    slide = openslide.OpenSlide(wsi_path)
    tissue_mask, mask_level = build_tissue_mask(slide)
    mask_scale = slide.level_downsamples[mask_level]
    target_downsample = slide.level_downsamples[target_level]

    extractor = ComprehensiveObjectFeatureExtractor()
    features_list = []
    width, height = slide.level_dimensions[target_level]

    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            x_level0 = int(x * target_downsample)
            y_level0 = int(y * target_downsample)
            patch_size_level0 = int(patch_size * target_downsample)

            ratio = tissue_ratio(tissue_mask, x_level0, y_level0, patch_size_level0, mask_scale)

            if ratio < tissue_threshold:
                continue

            patch = slide.read_region((x_level0, y_level0), target_level, (patch_size, patch_size))
            patch = np.array(patch)[:, :, :3]

            try:
                patch_features = extractor.extract_all_features(patch)
                patch_features['patch_x'] = x
                patch_features['patch_y'] = y
                patch_features['tissue_ratio'] = float(ratio)
                features_list.append(patch_features)
            except Exception:
                continue

    slide.close()
    return features_list

# ============================================================================
# STEP 6: SLIDE-LEVEL AGGREGATION
# ============================================================================

def aggregate_patch_features_to_slide_level(patch_features_list, slide_name):
    if not patch_features_list:
        return {}

    df = pd.DataFrame(patch_features_list)
    metadata_cols = ['patch_x', 'patch_y', 'tissue_ratio']
    feature_cols = [col for col in df.columns if col not in metadata_cols]

    slide_features = {'slide_name': slide_name, 'num_patches': len(patch_features_list)}

    for feature in feature_cols:
        feature_values = df[feature].dropna().values
        if len(feature_values) > 0:
            slide_features[f'{feature}_mean'] = float(np.mean(feature_values))
            slide_features[f'{feature}_std'] = float(np.std(feature_values))
            slide_features[f'{feature}_min'] = float(np.min(feature_values))
            slide_features[f'{feature}_max'] = float(np.max(feature_values))
            slide_features[f'{feature}_median'] = float(np.median(feature_values))
            slide_features[f'{feature}_q25'] = float(np.percentile(feature_values, 25))
            slide_features[f'{feature}_q75'] = float(np.percentile(feature_values, 75))
        else:
            for suffix in ['mean', 'std', 'min', 'max', 'median', 'q25', 'q75']:
                slide_features[f'{feature}_{suffix}'] = 0.0

    slide_features['tissue_ratio_mean'] = float(df['tissue_ratio'].mean())
    slide_features['tissue_ratio_std'] = float(df['tissue_ratio'].std())

    return slide_features

# ============================================================================
# STEP 7: PARALLEL SLIDE PROCESSING
# ============================================================================

def process_single_slide(args):
    slide_path, dataset_name, patch_size, target_level, tissue_threshold = args
    slide_name = os.path.basename(slide_path)
    try:
        patch_features = extract_object_features_from_wsi(
            slide_path, patch_size, target_level, tissue_threshold
        )
        if patch_features:
            slide_features = aggregate_patch_features_to_slide_level(
                patch_features, f"{dataset_name}_{slide_name}"
            )
            return {
                'success': True, 'slide_path': slide_path,
                'slide_name': slide_name, 'features': slide_features,
                'num_patches': len(patch_features), 'error': None
            }
        else:
            return {
                'success': False, 'slide_path': slide_path,
                'slide_name': slide_name, 'features': None,
                'num_patches': 0, 'error': 'No tissue patches found'
            }
    except Exception as e:
        return {
            'success': False, 'slide_path': slide_path,
            'slide_name': slide_name, 'features': None,
            'num_patches': 0, 'error': str(e)
        }

# ============================================================================
# STEP 8: MAIN PROCESSING LOOP
# ============================================================================

print("\n" + "="*80)
print("STARTING SLICED BATCH PROCESSING")
print("="*80)

# Build run-specific output folder so each run's CSV is separate
run_label = f"slides_{SLIDE_START_INDEX}_{SLIDE_END_INDEX - 1}"
output_path = f'/kaggle/working/object_features_{dataset_name}_{run_label}/'
os.makedirs(output_path, exist_ok=True)

# Collect and sort all slides so the index is deterministic across runs
all_slides = []
for root, dirs, files in os.walk(dataset_path):
    all_slides.extend([
        os.path.join(root, f) for f in files
        if f.endswith(('.svs', '.tif', '.tiff', '.ndpi'))
    ])
all_slides = sorted(all_slides)  # IMPORTANT: sort so index is consistent every run

print(f"Total slides in dataset: {len(all_slides)}")

# Apply the slice
slides_to_process = all_slides[SLIDE_START_INDEX:SLIDE_END_INDEX]
print(f"This run will process: {len(slides_to_process)} slides "
      f"(index {SLIDE_START_INDEX} → {SLIDE_END_INDEX - 1})")
for s in slides_to_process:
    print(f"  • {os.path.basename(s)}")

all_slides_features = []
total_processed = 0

for batch_start in range(0, len(slides_to_process), BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, len(slides_to_process))
    batch_slides = slides_to_process[batch_start:batch_end]

    print(f"\n{'='*80}")
    print(f"BATCH {batch_start // BATCH_SIZE + 1} — "
          f"slides {SLIDE_START_INDEX + batch_start + 1}–"
          f"{SLIDE_START_INDEX + batch_end} of {len(all_slides)} total")
    print(f"{'='*80}")

    slide_args = [(s, dataset_name, 256, 1, 0.3) for s in batch_slides]

    with ProcessPoolExecutor(max_workers=PARALLEL_WORKERS) as executor:
        futures = {executor.submit(process_single_slide, args): args[0] for args in slide_args}

        for future in as_completed(futures):
            slide_path = futures[future]
            try:
                result = future.result()
                if result['success']:
                    print(f"  ✓ {result['slide_name']}: {result['num_patches']} patches")
                    all_slides_features.append(result['features'])
                    total_processed += 1
                else:
                    print(f"  ✗ {result['slide_name']}: {result['error']}")
            except Exception as e:
                print(f"  ✗ Error with {os.path.basename(slide_path)}: {str(e)}")

    # Save interim CSV after every batch
    if all_slides_features:
        temp_df = pd.DataFrame(all_slides_features)
        temp_df.to_csv(f'{output_path}/slide_object_features_temp.csv', index=False)
        print(f"\n  💾 Batch checkpoint: {total_processed} slides saved so far")

# ============================================================================
# STEP 9: SAVE THIS RUN'S FINAL CSV
# ============================================================================

print("\n" + "="*80)
print("SAVING RUN RESULTS")
print("="*80)

if all_slides_features:
    final_df = pd.DataFrame(all_slides_features)
    print(f"Slides processed this run: {len(final_df)}")
    print(f"Features per slide: {final_df.shape[1]}")

    # Diagnostics
    nucleus_count_cols = [c for c in final_df.columns if 'nucleus_count' in c.lower()]
    for col in nucleus_count_cols:
        zero_count = (final_df[col] == 0).sum()
        print(f"{col}: mean={final_df[col].mean():.2f}, "
              f"zero slides={zero_count}/{len(final_df)} ({100*zero_count/len(final_df):.1f}%)")

    nan_total = final_df.isna().sum().sum()
    print(f"Total NaN values: {nan_total} ({'none ✓' if nan_total == 0 else 'check merge script'})")

    csv_path = f'{output_path}/{dataset_name}_{run_label}_features.csv'
    final_df.to_csv(csv_path, index=False)
    print(f"\n✓ Saved: {csv_path}")
else:
    print("⚠ No slides were successfully processed this run!")

# ============================================================================
# STEP 10: UPLOAD THIS RUN'S OUTPUT TO KAGGLE DATASETS
# ============================================================================

from kaggle import api

dataset_slug = f"object-features-{dataset_name.replace('_', '-')}-{run_label.replace('_', '-')}"
metadata = {
    "title": f"Object Features {dataset_name} {run_label}",
    "id": f"onyekamuoka/{dataset_slug}",
    "licenses": [{"name": "CC0-1.0"}]
}

with open(f'{output_path}/dataset-metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

try:
    api.dataset_create_new(
        folder=output_path, dir_mode='zip', convert_to_csv=False, public=False
    )
    print(f"✓ Created Kaggle dataset: {dataset_slug}")
except Exception:
    api.dataset_create_version(
        folder=output_path,
        version_notes=f"Run {run_label}",
        dir_mode='zip', convert_to_csv=False
    )
    print(f"✓ Updated Kaggle dataset: {dataset_slug}")

print(f"\n✓ https://www.kaggle.com/datasets/onyekamuoka/{dataset_slug}")
print("\n" + "="*80)
print(f"RUN COMPLETE — {run_label}")
print("Change SLIDE_START_INDEX and SLIDE_END_INDEX and run again for next slice.")
print("When all runs done, use bracs_wsi_merge.py to combine all CSVs.")
print("="*80)